## **NGC 1300 - Galaxia barrada**

Base de código para el cálculo, en las imágenes del HST, de: <br>
- mosaico (CREACIÓN DEL MOSAICO) 
- centroide (MÁSCARA PARA NÚCLEO)
- transformación XY - V2V3 - NE (TRANSFORMACIÓN DE COORDENADAS)
- visualización con todo lo anterior, orientación, retícula, ejes... (VISUALIZACIÓN)


In [ ]:
# Librerías necesarias
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib.colors import LogNorm
from astropy.wcs import WCS
import math
from astropy import units as u
from scipy.ndimage import label, center_of_mass
from skimage.measure import regionprops

# Cargar la fuente
font_path = "/usr/share/fonts/msttcore/times.ttf"
tnr_font = fm.FontProperties(fname=font_path)

###############################  CREACIÓN DEL MOSAICO   ################################################
# Lista de archivos FITS a combinar
archivos = ['NGC1300/NGC1300_HST/idxr06030_drz.fits', 'NGC1300/NGC1300_HST/idxr07030_drz.fits']
imagenes = []
wcs_list = []

# Cargar datos e información WCS
for archivo in archivos:
    hdu = fits.open(archivo)
    data = np.nan_to_num(hdu[1].data)  # Reemplazar NaN por 0
    wcs = WCS(hdu[1].header)
    hdu.close()
    imagenes.append(data)
    wcs_list.append(wcs)

# Generar WCS y forma óptima para el mosaico
wcs_out, shape_out = find_optimal_celestial_wcs([(imagenes[i], wcs_list[i]) for i in range(len(imagenes))])

# Crear mosaico con reproject
mosaico, footprint = reproject_and_coadd(
    [(imagenes[i], wcs_list[i]) for i in range(len(imagenes))],
    wcs_out, shape_out=shape_out, reproject_function=reproject_interp
)

# Obtener dimensiones
height, width = mosaico.shape
print(f"Píxeles del mosaico: {mosaico.shape}")


###################### PASO DE AR - h m s Y DEC º ' " ######################
# Función para convertir grados a horas, minutos, segundos (1h son 15º)
def grados_a_hms(ar_deg):
    horas = int(ar_deg / 15)  # Convertir grados a horas
    minutos = int((ar_deg / 15 - horas) * 60)  # Convertir la parte fraccionaria de horas a minutos
    segundos = ((ar_deg / 15 - horas) * 60 - minutos) * 60  # Convertir la parte fraccionaria de minutos a segundos
    return f"{horas}h {minutos}m {segundos:.2f}s"
# Convertir AR a formato de horas, minutos y segundos
#AR_hms = grados_a_hms(AR)

# Función para convertir grados a grados, minutos y segundos
def grados_a_gms(deg):
    grados = int(deg)  # Parte entera de los grados
    minutos = int(abs(deg - grados) * 60)  # Convertir la parte decimal de grados a minutos
    segundos = (abs(deg - grados) * 60 - minutos) * 60  # Convertir la parte decimal de minutos a segundos
    return f"{grados}° {minutos}' {segundos:.2f}'' "

# Convertir DEC a formato de grados, minutos y segundos
#DEC_gms = grados_a_gms(DEC)

##################################################################

#####################################   MÁSCARA PARA NÚCLEO   #####################################

#  Definir el radio de interés en píxeles para el núcleo
R_nucleo = 1550  # Reducimos el radio para centrarnos más en la zona brillante

#  Obtener el centro de la imagen
center_x, center_y = height // 2, width // 2

#  Crear una máscara circular para limitar la búsqueda del núcleo
Y, X = np.ogrid[:height, :width]
mask_circle_nucleo = (X - center_x) ** 2 + (Y - center_y) ** 2 <= R_nucleo ** 2  # Ecuación del círculo

#  Aplicar la máscara circular a la imagen original
masked_mosaico = np.where(mask_circle_nucleo, mosaico, 0)

#  Aplicar un umbral de brillo SOLO dentro del círculo del núcleo
umbral_nucleo = np.percentile(masked_mosaico[mask_circle_nucleo], 95)  # Subimos el umbral para mejor precisión
mask_nucleo = masked_mosaico > umbral_nucleo

#  Encontrar las regiones conectadas del núcleo
labeled, num_features = label(mask_nucleo)
region_sizes = np.bincount(labeled.ravel())[1:]  # Contar tamaños de regiones
largest_region_idx = np.argmax(region_sizes) + 1  # Índice de la región más grande
mask_nucleo = labeled == largest_region_idx  # Filtrar solo la región central

#  Obtener el centroide ponderado (núcleo de la nebulosa)
centroid = center_of_mass(masked_mosaico * mask_nucleo)
centroid = (int(centroid[0]), int(centroid[1]))  # Convertir a enteros
print('---------------------------------------------------------------------------')
print(f"Centro refinado de NGC 1300: {centroid}")

# Convertir a coordenadas celestes
RA_centroid, DEC_centroid = wcs.all_pix2world([[centroid[1], centroid[0]]], 1)[0]
RA_centroid_hms = grados_a_hms(RA_centroid)
DEC_centroid_gms = grados_a_gms(DEC_centroid)
print(f"Centro refinado de NGC 1300 en coordendas (α,δ): ({RA_centroid_hms}, {DEC_centroid_gms}) ")
print('----------------------------------------------------------------------------')

#####################################   TRANSFORMACIÓN DE COORDENADAS   #####################################
# Datos de entrada
Xr = 2097.0
Yr = 2046.0
V2r = 0.4314  # en arcosegundos
V3r = -3.7840 # en arcosegundos
Sx = 0.039750
Sy = 0.039591
beta_x = -41.4701  # en grados
beta_y = 44.7597   # en grados
X = 3517.0
Y = 4360.0
# Aunque sea un mosaico, de dos imgs con mismo filtro, cada uno tiene su valor de orientación:
PA_V3_img1   = 247.007095 # 6030
PA_V3_img2  = 246.993698 # 7030
# Así que haremos la media
PA_V3 = (PA_V3_img1 + PA_V3_img2) / 2 # grados

# Conversión de grados a radianes, para poder trabajar con ellos
beta_x_rad = math.radians(beta_x)
beta_y_rad = math.radians(beta_y)

# Convertir PA_V3 a radianes
PA_V3_rad = math.radians(PA_V3)

# Cálculo de V2 arcosegundos
V2 = V2r + Sx * math.sin(beta_x_rad) * (X - Xr) + Sy * math.sin(beta_y_rad) * (Y - Yr)

# Cálculo de V3 arcosegundos
V3 = V3r + Sx * math.cos(beta_x_rad) * (X - Xr) + Sy * math.cos(beta_y_rad) * (Y - Yr)

# Rotación de las coordenadas V2 y V3 al sistema Norte y Este
N = V3 * math.cos(PA_V3_rad) - V2 * math.sin(PA_V3_rad)  # Coordenada Norte relativa
E = V3 * math.sin(PA_V3_rad) + V2 * math.cos(PA_V3_rad)  # Coordenada Este relativa

print('----------------------------------------------------------------------------')
print('')
#####################################   VISUALIZACIÓN   #####################################
vmin = np.percentile(mosaico[mosaico > 0], 30)   # 5% más oscuro
vmax = np.percentile(mosaico[mosaico > 0], 99.9) # 99.5% más brillante

# Crear la figura sin distorsiones
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(projection=wcs)  ## Trabajar en 2D

#  Ajuste del colormap y el contraste
im = ax.imshow(mosaico, cmap='inferno', norm=LogNorm(vmin=vmin, vmax=vmax))  # Ajustado para más contraste
ax.set_title('NGC 1300 - Galaxia espiral barrada (F336W)', fontproperties=tnr_font, fontsize=16, pad=15)
ax.set_xlabel('δ', fontproperties=tnr_font, fontsize=12)
ax.set_ylabel('α', fontproperties=tnr_font, fontsize=12)

# Marcar el centro refinado en la imagen
ax.plot(centroid[1], centroid[0], '+', color='black', markersize=7, label="Centro Refinado")

# Graficar la línea de envergadura ajustada con máscara circular y SIMÉTRICA
#ax.plot([x_min, x_max], [row, row], color='cyan', linewidth=2, label="Envergadura de la nebulosa")

# Añadir anotación para AR y DEC
text_str = f"Coordenadas del núcleo\nα: {grados_a_hms(RA_centroid)}\nδ: {grados_a_gms(DEC_centroid)}"
ax.text(0.95, 0.95, text_str, transform=ax.transAxes, color='white', fontsize=10,
        fontweight='bold', fontproperties=tnr_font, ha='right', va='top', bbox=dict(facecolor='black', alpha=0.5))

# Añadir anotación para N y E
#text_str = f"Nc: {N:.6f} (arcsec)\nEc: {E:.6f} (arcsec)"
#ax.text(0.95, 0.849, text_str, transform=ax.transAxes, color='white', fontsize=10,
#        fontweight='bold', fontproperties=tnr_font, ha='right', va='top', bbox=dict(facecolor='black', alpha=0.5))

# Etiquetar la envergadura
#text_str = f"Envergadura: {x_max - x_min:.2f} píxeles"
#ax.text(0.95, 0.82, text_str, transform=ax.transAxes, color='white', fontsize=10,
#        fontweight='bold', fontproperties=tnr_font, ha='right', va='top', bbox=dict(facecolor='black', alpha=0.5))

# --------------------------
# Añadir Norte y Este en la retícula correcta
# Punto en la esquina inferior derecha para la orientación
x, y = 7885, 170.5
ra_dec_corner = wcs.pixel_to_world(x, y)

# Desplazamientos para las flechas
delta_ra = (0.01 * u.deg)
delta_dec = (0.01 * u.deg)

# Calcular coordenadas de Norte y Este
ra_dec_norte = ra_dec_corner.spherical_offsets_by(0 * u.deg, delta_dec)
ra_dec_este = ra_dec_corner.spherical_offsets_by(delta_ra, 0 * u.deg)

# Convertir de vuelta a píxeles
norte_x, norte_y = wcs.world_to_pixel(ra_dec_norte)
este_x, este_y = wcs.world_to_pixel(ra_dec_este)

# Dibujar flechas en la dirección correcta
ax.arrow(x, y, norte_x - x, norte_y - y, 
         color='black', head_width=35, head_length=30, label="Norte")
ax.arrow(x, y, este_x - x, este_y - y, 
         color='black', head_width=35, head_length=30, label="Este")

# Etiquetas
ax.text(norte_x, norte_y + 15, 'N', color='black', fontsize=13, fontproperties=tnr_font, ha='center', va='bottom')
ax.text(este_x + 90, este_y - 140, 'E', color='black', fontsize=13, fontproperties=tnr_font, ha='center', va='bottom')


# Añadir la barra de color
cbar = plt.colorbar(im, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)
cbar.set_label('Brillo', fontproperties=tnr_font, fontsize=12)
cbar.ax.tick_params(labelsize=10)
for label in cbar.ax.get_yticklabels():
    label.set_fontproperties(tnr_font)


#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax.coords[0].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # α (RA)
ax.coords[1].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon = ax.coords[0]  # Eje de Ascensión Recta (α)
lat = ax.coords[1]  # Eje de Declinación (δ)


# Forzar la visualización completa de horas, minutos y segundos en α
lon.set_major_formatter('hh:mm:ss.s')

# Forzar la visualización completa de grados, minutos y segundos en δ
lat.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidadno
lon.display_minor_ticks(True)
lat.display_minor_ticks(True)
#############################################################################



# Configurar la retícula correctamente
ax.grid(color='black', linestyle='--', linewidth=0.5)
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)  # Ajuste del espacio

plt.show()


Base de código para el cálculo, en las imágenes del HST, de: <br>
- centroide (MÁSCARA PARA NÚCLEO)
- transformación XY - V2V3 - NE (TRANSFORMACIÓN DE COORDENADAS)
- estrellas brillantes (DETECCIÓN DE ESTRELLAS BRILLANTES)
- distancia del centroide a las estrellas más brillants (DISTANCIAS FÍSICAS A ESTRELLAS BRILLANTES)
- detección de las diferentes zonas de la galaxia (SECCIONADO DE GALAXIA)
- visualización con todo lo anterior, orientación, retícula, ejes... (VISUALIZACIÓN)


In [ ]:
# Librerías necesarias
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib.colors import LogNorm
from astropy.wcs import WCS
import math
from astropy import units as u
from scipy.ndimage import label, center_of_mass
from skimage.measure import regionprops
from matplotlib.patches import Circle
from skimage.feature import peak_local_max

# Cargar la fuente
font_path = "/usr/share/fonts/msttcore/times.ttf"
tnr_font = fm.FontProperties(fname=font_path)

# Cargar los datos de la imagen
F814W = fits.open('NGC1300/NGC1300_HST/idkv06010_drz.fits')  # FITS file
F814W_h = F814W[0].header
F814W_d = fits.getdata('NGC1300/NGC1300_HST/idkv06010_drz.fits')
F814W_d = np.nan_to_num(F814W_d)  # Reemplazar NaN por 0

# Obtener dimensiones
height, width = F814W_d.shape
print(f"Píxeles de la imagen: {F814W_d.shape}")

# Obtener WCS
wcs = WCS(F814W[1].header, naxis=2)

###################### PASO DE AR - h m s Y DEC º ' " ######################
# Obtener AR y DEC de la cabecera
AR = F814W_h['RA_TARG']
DEC = F814W_h['DEC_TARG']
# Función para convertir grados a horas, minutos, segundos (1h son 15º)
def grados_a_hms(ar_deg):
    horas = int(ar_deg / 15)  # Convertir grados a horas
    minutos = int((ar_deg / 15 - horas) * 60)  # Convertir la parte fraccionaria de horas a minutos
    segundos = ((ar_deg / 15 - horas) * 60 - minutos) * 60  # Convertir la parte fraccionaria de minutos a segundos
    return f"{horas}h {minutos}m {segundos:.2f}s"
# Convertir AR a formato de horas, minutos y segundos
AR_hms = grados_a_hms(AR)

# Función para convertir grados a grados, minutos y segundos
def grados_a_gms(deg):
    grados = int(deg)  # Parte entera de los grados
    minutos = int(abs(deg - grados) * 60)  # Convertir la parte decimal de grados a minutos
    segundos = (abs(deg - grados) * 60 - minutos) * 60  # Convertir la parte decimal de minutos a segundos
    return f"{grados}° {minutos}' {segundos:.2f}'' "

# Convertir DEC a formato de grados, minutos y segundos
DEC_gms = grados_a_gms(DEC)

##################################################################

#####################################   MÁSCARA PARA NÚCLEO   #####################################

#  Definir el radio de interés en píxeles para el núcleo
R_nucleo = 450  # Reducimos el radio para centrarnos más en la zona brillante

#  Obtener el centro de la imagen
center_x, center_y = height // 2, width // 2

#  Crear una máscara circular para limitar la búsqueda del núcleo
Y, X = np.ogrid[:height, :width]
mask_circle_nucleo = (X - center_x) ** 2 + (Y - center_y) ** 2 <= R_nucleo ** 2  # Ecuación del círculo

#  Aplicar la máscara circular a la imagen original
masked_F814W = np.where(mask_circle_nucleo, F814W_d, 0)

#  Aplicar un umbral de brillo SOLO dentro del círculo del núcleo
umbral_nucleo = np.percentile(masked_F814W[mask_circle_nucleo], 75)  # Subimos el umbral para mejor precisión
mask_nucleo = masked_F814W > umbral_nucleo

#  Encontrar las regiones conectadas del núcleo
labeled, num_features = label(mask_nucleo)
region_sizes = np.bincount(labeled.ravel())[1:]  # Contar tamaños de regiones
largest_region_idx = np.argmax(region_sizes) + 1  # Índice de la región más grande
mask_nucleo = labeled == largest_region_idx  # Filtrar solo la región central

#  Obtener el centroide ponderado (núcleo de la nebulosa)
centroid = center_of_mass(masked_F814W * mask_nucleo)
centroid = (int(centroid[0]), int(centroid[1]))  # Convertir a enteros
print('---------------------------------------------------------------------------')
print(f"Centro refinado de NGC 1300: {centroid}")

# Convertir a coordenadas celestes
RA_centroid, DEC_centroid = wcs.all_pix2world([[centroid[1], centroid[0]]], 1)[0]
RA_centroid_hms = grados_a_hms(RA_centroid)
DEC_centroid_gms = grados_a_gms(DEC_centroid)
print(f"Centro refinado de NGC 1300 en coordendas (α,δ): ({RA_centroid_hms}, {DEC_centroid_gms}) ")
print('----------------------------------------------------------------------------')


#####################################   TRANSFORMACIÓN DE COORDENADAS   #####################################
# Datos de entrada
Xr = 537.0
Yr = 512.0
V2r = -1.2858  # en arcosegundos
V3r = -93.3875 # en arcosegundos
Sx = 0.039568
Sy = 0.039802
beta_x = -41.7091  # en grados
beta_y = 44.5071   # en grados
X = 538.0
Y = 500.0
PA_V3 = 148.169800  # Ángulo PA_V3 en grados

# Conversión de grados a radianes, para poder trabajar con ellos
beta_x_rad = math.radians(beta_x)
beta_y_rad = math.radians(beta_y)

# Convertir PA_V3 a radianes
PA_V3_rad = math.radians(PA_V3)

# Cálculo de V2 arcosegundos
V2 = V2r + Sx * math.sin(beta_x_rad) * (X - Xr) + Sy * math.sin(beta_y_rad) * (Y - Yr)

# Cálculo de V3 arcosegundos
V3 = V3r + Sx * math.cos(beta_x_rad) * (X - Xr) + Sy * math.cos(beta_y_rad) * (Y - Yr)

# Rotación de las coordenadas V2 y V3 al sistema Norte y Este
N = V3 * math.cos(PA_V3_rad) - V2 * math.sin(PA_V3_rad)  # Coordenada Norte relativa
E = V3 * math.sin(PA_V3_rad) + V2 * math.cos(PA_V3_rad)  # Coordenada Este relativa

print('----------------------------------------------------------------------------')
print('')
#####################################   DETECCIÓN DE ESTRELLAS BRILLANTES   #####################################
# Paso 1: Normalización (si no la tienes ya hecha)
img_norm = (F814W_d - np.min(F814W_d)) / (np.max(F814W_d) - np.min(F814W_d))

# Paso 2: Detección de picos locales (estrellas brillantes)
coords_estrellas = peak_local_max(img_norm, min_distance=5, threshold_abs=0.4)  # Puedes ajustar el umbral

# Mensaje informativo
print(f"Se han detectado y marcado las {len(coords_estrellas)} estrellas más brillantes.")

#####################################   DISTANCIAS FÍSICAS A ESTRELLAS BRILLANTES   #####################################

# Distancia a NGC 1300 en años luz
d_ly = 61000000 # información sacada de internet
ly_to_pc = 0.3066
d_pc = d_ly * ly_to_pc  

# Escala de píxel (arcsec/pix)
pixel_scales = np.abs(wcs.pixel_scale_matrix.diagonal()) * 3600
arcsec_per_pix = np.mean(pixel_scales)

# Calcular distancias en píxeles desde el centroide
pix_distances = [np.linalg.norm(np.array([y, x]) - np.array(centroid)) for y, x in coords_estrellas]

# Pasar a arcsec
arcsec_distances = np.array(pix_distances) * arcsec_per_pix

# Pasar a radianes
theta_rad = (arcsec_distances / 3600) * (np.pi / 180)

# Distancias físicas en parsec
pc_distances = theta_rad * d_pc

# Mostrar 5 resultados más representativos (puedes ajustar el número)
print('\nDistancias desde el núcleo a las estrellas más brillantes:')
for i, (p, a, pc) in enumerate(zip(pix_distances[:5], arcsec_distances[:5], pc_distances[:5]), 1):
    print(f"Estrella {i}: {p:.1f} px ≈ {a:.2f}\" ≈ {pc:.2f} pc")



#####################################   SECCIONADO DE GALAXIA   #######################################
# Calcular vmin y vmax globales para LogNorm
vmin = np.percentile(F814W_d[F814W_d > 0], 30)   # 5% más oscuro
vmax = np.percentile(F814W_d[F814W_d > 0], 99.9) # 99.5% más brillante

# --- Suavizar imagen para destacar regiones brillantes ---
F814W_smooth = gaussian_filter(F814W_d, sigma=6)  # Ajusta sigma según nivel de suavizado deseado

# --- Definir niveles de contorno ---
levels = np.logspace(np.log10(vmin), np.log10(vmax), num=6)  # Número de contornos

#####################################   VISUALIZACIÓN   #####################################
#vmin = np.percentile(F814W_d[F814W_d > 0], 30)   # 5% más oscuro
#vmax = np.percentile(F814W_d[F814W_d > 0], 99.9) # 99.5% más brillante
# Crear la figura sin distorsiones
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(projection=wcs)  ## Trabajar en 2D

#  Ajuste del colormap y el contraste
im = ax.imshow(F814W_d, cmap='inferno', norm=LogNorm(vmin=vmin, vmax=vmax))  # Ajustado para más contraste
# --- Dibujar contornos que resalten las regiones ---
ax.contour(F814W_smooth, levels=levels, colors='cyan', linewidths=1.2, alpha=0.7)

ax.set_title('NGC 1300 - Galaxia espiral barrada (F814W)', fontproperties=tnr_font, fontsize=16, pad=15)
ax.set_xlabel('α', fontproperties=tnr_font, fontsize=12)
ax.set_ylabel('δ', fontproperties=tnr_font, fontsize=12)
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax.coords[0].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # α (RA)
ax.coords[1].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Marcar el centro refinado en la imagen
ax.plot(centroid[1], centroid[0], '+', color='black', markersize=7, label="Centro Refinado")

# Dibujar círculos estrellas más brillantes y líneas desde el núcleo
for i, (y, x) in enumerate(coords_estrellas[:5]):  # solo las 5 más cercanas por claridad
    ax.add_patch(Circle((x, y), radius=10, edgecolor='lime', facecolor='none', linewidth=1.6))
    ax.plot([centroid[1], x], [centroid[0], y], color='orange', linestyle='--', linewidth=1.2)
    ax.text(x + 12, y, f'{i+1}', color='white', fontsize=13, fontproperties=tnr_font,
            ha='left', va='center', bbox=dict(facecolor='black', alpha=0.5, pad=1.5))


# Añadir anotación para AR y DEC
text_str = f"Coordenadas del núcleo\nα: {grados_a_hms(RA_centroid)}\nδ: {grados_a_gms(DEC_centroid)}"
ax.text(0.95, 0.95, text_str, transform=ax.transAxes, color='white', fontsize=10,
        fontweight='bold', fontproperties=tnr_font, ha='right', va='top', bbox=dict(facecolor='black', alpha=0.5))

# Añadir anotación para N y E
#text_str = f"Nc: {N:.6f} (arcsec)\nEc: {E:.6f} (arcsec)"
#ax.text(0.95, 0.855, text_str, transform=ax.transAxes, color='white', fontsize=10,
#        fontweight='bold', fontproperties=tnr_font, ha='right', va='top', bbox=dict(facecolor='black', alpha=0.5))



# --------------------------
# Añadir Norte y Este en la retícula correcta
# Punto en la esquina inferior derecha para la orientación
x, y = 850, 170.5
ra_dec_corner = wcs.pixel_to_world(x, y)

# Desplazamientos para las flechas
delta_ra = (0.001 * u.deg)
delta_dec = (0.001 * u.deg)

# Calcular coordenadas de Norte y Este
ra_dec_norte = ra_dec_corner.spherical_offsets_by(0 * u.deg, delta_dec)
ra_dec_este = ra_dec_corner.spherical_offsets_by(delta_ra, 0 * u.deg)

# Convertir de vuelta a píxeles
norte_x, norte_y = wcs.world_to_pixel(ra_dec_norte)
este_x, este_y = wcs.world_to_pixel(ra_dec_este)

# Dibujar flechas en la dirección correcta
ax.arrow(x, y, norte_x - x, norte_y - y, 
         color='black', head_width=20, head_length=15, label="Norte")
ax.arrow(x, y, este_x - x, este_y - y, 
         color='black', head_width=20, head_length=15, label="Este")

# Etiquetas
ax.text(norte_x - 20, norte_y, 'N', color='black', fontsize=13, fontproperties=tnr_font, ha='center', va='bottom')
ax.text(este_x, este_y + 10, 'E', color='black', fontsize=13, fontproperties=tnr_font, ha='center', va='bottom')


# Añadir la barra de color
cbar = plt.colorbar(im, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)
cbar.set_label('Brillo', fontproperties=tnr_font, fontsize=12)
cbar.ax.tick_params(labelsize=10)
for label in cbar.ax.get_yticklabels():
    label.set_fontproperties(tnr_font)


# Configurar la retícula correctamente
ax.grid(color='gray', linestyle='--', linewidth=0.6)
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)  # Ajuste del espacio

plt.show()
F814W.close()


Base de código para el cálculo, en las imágenes del JWST, de: <br>
- centroide (MÁSCARA PARA NÚCLEO)
- transformación XY - V2V3 - NE (TRANSFORMACIÓN DE COORDENADAS)
- estrellas brillantes (DETECCIÓN DE ESTRELLAS BRILLANTES)
- distancia del centroide a las estrellas más brillants (DISTANCIAS FÍSICAS A ESTRELLAS BRILLANTES)
- visualización con todo lo anterior, orientación, retícula, ejes... (VISUALIZACIÓN)


In [ ]:
# Librerías necesarias
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib.colors import LogNorm
from astropy.wcs import WCS
import math
from astropy import units as u
from scipy.ndimage import label, center_of_mass
from skimage.measure import regionprops
from astropy.visualization import ImageNormalize, AsinhStretch

# Cargar la fuente
font_path = "/usr/share/fonts/msttcore/times.ttf"
tnr_font = fm.FontProperties(fname=font_path)

# Cargar los datos de la imagen
F1000W = fits.open('NGC1300/NGC1300_JWST/jw02107-o002_t002_miri_f1000w_i2d.fits')  # FITS file
F1000W_h = F1000W[0].header
F1000W_d = fits.getdata('NGC1300/NGC1300_JWST/jw02107-o002_t002_miri_f1000w_i2d.fits')
F1000W_d = np.nan_to_num(F1000W_d)  # Reemplazar NaN por 0

# Obtener dimensiones
height, width = F1000W_d.shape
print(f"Píxeles de la imagen: {F1000W_d.shape}")

# Obtener WCS
wcs = WCS(F1000W[1].header, naxis=2)

###################### PASO DE AR - h m s Y DEC º ' " ######################
# Obtener AR y DEC de la cabecera
AR = F1000W_h['TARG_RA']
DEC = F1000W_h['TARG_DEC']
# Función para convertir grados a horas, minutos, segundos (1h son 15º)
def grados_a_hms(ar_deg):
    horas = int(ar_deg / 15)  # Convertir grados a horas
    minutos = int((ar_deg / 15 - horas) * 60)  # Convertir la parte fraccionaria de horas a minutos
    segundos = ((ar_deg / 15 - horas) * 60 - minutos) * 60  # Convertir la parte fraccionaria de minutos a segundos
    return f"{horas}h {minutos}m {segundos:.2f}s"
# Convertir AR a formato de horas, minutos y segundos
AR_hms = grados_a_hms(AR)

# Función para convertir grados a grados, minutos y segundos
def grados_a_gms(deg):
    grados = int(deg)  # Parte entera de los grados
    minutos = int(abs(deg - grados) * 60)  # Convertir la parte decimal de grados a minutos
    segundos = (abs(deg - grados) * 60 - minutos) * 60  # Convertir la parte decimal de minutos a segundos
    return f"{grados}° {minutos}' {segundos:.2f}'' "

# Convertir DEC a formato de grados, minutos y segundos
DEC_gms = grados_a_gms(DEC)

##################################################################

#####################################   MÁSCARA PARA NÚCLEO   #####################################

#  Definir el radio de interés en píxeles para el núcleo
R_nucleo = 500  # Reducimos el radio para centrarnos más en la zona brillante

#  Obtener el centro de la imagen
center_x, center_y = height // 2, width // 2

#  Crear una máscara circular para limitar la búsqueda del núcleo
Y, X = np.ogrid[:height, :width]
mask_circle_nucleo = (X - center_x) ** 2 + (Y - center_y) ** 2 <= R_nucleo ** 2  # Ecuación del círculo

#  Aplicar la máscara circular a la imagen original
masked_F1000W = np.where(mask_circle_nucleo, F1000W_d, 0)

#  Aplicar un umbral de brillo SOLO dentro del círculo del núcleo
umbral_nucleo = np.percentile(masked_F1000W[mask_circle_nucleo], 93)  # Subimos el umbral para mejor precisión
mask_nucleo = masked_F1000W > umbral_nucleo

#  Encontrar las regiones conectadas del núcleo
labeled, num_features = label(mask_nucleo)
region_sizes = np.bincount(labeled.ravel())[1:]  # Contar tamaños de regiones
largest_region_idx = np.argmax(region_sizes) + 1  # Índice de la región más grande
mask_nucleo = labeled == largest_region_idx  # Filtrar solo la región central

#  Obtener el centroide ponderado (núcleo de la nebulosa)
centroid = center_of_mass(masked_F1000W * mask_nucleo)
centroid = (int(centroid[0]), int(centroid[1]))  # Convertir a enteros
print('---------------------------------------------------------------------------')
print(f"Centro refinado de NGC 1300: {centroid}")

# Convertir a coordenadas celestes
RA_centroid, DEC_centroid = wcs.all_pix2world([[centroid[1], centroid[0]]], 1)[0]
RA_centroid_hms = grados_a_hms(RA_centroid)
DEC_centroid_gms = grados_a_gms(DEC_centroid)
print(f"Centro refinado de NGC 1300 en coordendas (α,δ): ({RA_centroid_hms}, {DEC_centroid_gms}) ")
print('----------------------------------------------------------------------------')

#####################################   TRANSFORMACIÓN DE COORDENADAS   #####################################
# Datos de entrada
Xr = 693.5
Yr = 512.5
V2r = -453.37849  # en arcosegundos
V3r = -373.810549 # en arcosegundos

# Calcular escala de píxel (en grados/píxel → luego a arcsec/píxel)
pixel_scales_deg = wcs.pixel_scale_matrix.diagonal()
pixel_scale = np.abs(pixel_scales_deg) * 3600  # en arcsec/píxel
Sx = float(pixel_scale[0])
Sy = float(pixel_scale[1])

# Se dan por hecho en el JWST por la alineación de los instrumentos
beta_x = 0  # en grados
beta_y = 90  # en grados
X = 1040.0  
Y = 1035.0
PA_V3 = 69.64037886677356  # Ángulo PA_V3 en grados

# Conversión de grados a radianes, para poder trabajar con ellos
beta_x_rad = math.radians(beta_x)
beta_y_rad = math.radians(beta_y)

# Convertir PA_V3 a radianes
PA_V3_rad = math.radians(PA_V3)

# Cálculo de V2 arcosegundos
V2 = V2r + Sx * math.sin(beta_x_rad) * (X - Xr) + Sy * math.sin(beta_y_rad) * (Y - Yr)

# Cálculo de V3 arcosegundos
V3 = V3r + Sx * math.cos(beta_x_rad) * (X - Xr) + Sy * math.cos(beta_y_rad) * (Y - Yr)

# Rotación de las coordenadas V2 y V3 al sistema Norte y Este
N = V3 * math.cos(PA_V3_rad) - V2 * math.sin(PA_V3_rad)  # Coordenada Norte relativa
E = V3 * math.sin(PA_V3_rad) + V2 * math.cos(PA_V3_rad)  # Coordenada Este relativa

print('----------------------------------------------------------------------------')
print('')

#####################################   DETECCIÓN DE ESTRELLAS BRILLANTES   #####################################
# Paso 1: Normalización (si no la tienes ya hecha)
img_norm = (F1000W_d - np.min(F1000W_d)) / (np.max(F1000W_d) - np.min(F1000W_d))

# Paso 2: Detección de picos locales (estrellas brillantes)
coords_estrellas = peak_local_max(img_norm, min_distance=5, threshold_abs=0.2)  # Puedes ajustar el umbral

# Mensaje informativo
print(f"Se han detectado y marcado las {len(coords_estrellas)} estrellas más brillantes.")

#####################################   DISTANCIAS FÍSICAS A ESTRELLAS BRILLANTES (NGC 1300)   #####################################

# Distancia a NGC 1300 en años luz
d_ly = 61000000 # información sacada de internet
ly_to_pc = 0.3066
d_pc = d_ly * ly_to_pc  

# Escala de píxel (arcsec/pix)
pixel_scales = np.abs(wcs.pixel_scale_matrix.diagonal()) * 3600
arcsec_per_pix = np.mean(pixel_scales)

# Calcular distancias en píxeles desde el centroide
pix_distances = [np.linalg.norm(np.array([y, x]) - np.array(centroid)) for y, x in coords_estrellas]

# Pasar a arcsec
arcsec_distances = np.array(pix_distances) * arcsec_per_pix

# Pasar a radianes
theta_rad = (arcsec_distances / 3600) * (np.pi / 180)

# Distancias físicas en parsec
pc_distances = theta_rad * d_pc

# Mostrar 5 resultados más representativos (puedes ajustar el número)
print('\nDistancias desde el núcleo a las estrellas más brillantes:')
for i, (p, a, pc) in enumerate(zip(pix_distances[:5], arcsec_distances[:5], pc_distances[:5]), 1):
    print(f"Estrella {i}: {p:.1f} px ≈ {a:.2f}\" ≈ {pc:.2f} pc")


#####################################   VISUALIZACIÓN   #####################################
vmin = np.percentile(F1000W_d[F1000W_d > 0], 30)   # 5% más oscuro
vmax = np.percentile(F1000W_d[F1000W_d > 0], 99.9) # 99.5% más brillante
# Crear la figura sin distorsiones
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(projection=wcs)  ## Trabajar en 2D

#  Ajuste del colormap y el contraste
im = ax.imshow(F1000W_d, cmap='inferno', norm= ImageNormalize(F1000W_d, stretch=AsinhStretch(), vmin=vmin, vmax=vmax))  # Ajustado para más contraste
ax.set_title('NGC 1300 - Galaxia espiral barrada (F1000W)', fontproperties=tnr_font, fontsize=16, pad=15)
ax.set_xlabel('δ', fontproperties=tnr_font, fontsize=12)
ax.set_ylabel('α', fontproperties=tnr_font, fontsize=12)
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax.coords[0].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # α (RA)
ax.coords[1].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Marcar el centro refinado en la imagen
ax.plot(centroid[1], centroid[0], '+', color='black', markersize=7, label="Centro Refinado")

# Dibujar círculos estrellas más brillantes y líneas desde el núcleo
for i, (y, x) in enumerate(coords_estrellas[:5]):  # solo las 5 más cercanas por claridad
    ax.add_patch(Circle((x, y), radius=10, edgecolor='lime', facecolor='none', linewidth=1.6))
    ax.plot([centroid[1], x], [centroid[0], y], color='orange', linestyle='--', linewidth=1.2)
    ax.text(x + 12, y, f'{i+1}', color='white', fontsize=13, fontproperties=tnr_font,
            ha='left', va='center', bbox=dict(facecolor='black', alpha=0.5, pad=1.5))

# Añadir anotación para AR y DEC
text_str = f"Coordenadas del núcleo\nα: {grados_a_hms(RA_centroid)}\nδ: {grados_a_gms(DEC_centroid)}"
ax.text(0.95, 0.95, text_str, transform=ax.transAxes, color='white', fontsize=12,
        fontweight='bold', fontproperties=tnr_font, ha='right', va='top', bbox=dict(facecolor='black', alpha=0.5))

# Añadir anotación para N y E
#text_str = f"Nc: {N:.6f} (arcsec)\nEc: {E:.6f} (arcsec)"
#ax.text(0.95, 0.855, text_str, transform=ax.transAxes, color='white', fontsize=10,
#        fontweight='bold', fontproperties=tnr_font, ha='right', va='top', bbox=dict(facecolor='black', alpha=0.5))

# --------------------------
# Añadir Norte y Este en la retícula correcta
# Punto en la esquina inferior derecha para la orientación
x, y = 1500, 150.5
ra_dec_corner = wcs.pixel_to_world(x, y)

# Desplazamientos para las flechas
delta_ra = (0.003 * u.deg)
delta_dec = (0.003 * u.deg)

# Calcular coordenadas de Norte y Este
ra_dec_norte = ra_dec_corner.spherical_offsets_by(0 * u.deg, delta_dec)
ra_dec_este = ra_dec_corner.spherical_offsets_by(delta_ra, 0 * u.deg)

# Convertir de vuelta a píxeles
norte_x, norte_y = wcs.world_to_pixel(ra_dec_norte)
este_x, este_y = wcs.world_to_pixel(ra_dec_este)

# Dibujar flechas en la dirección correcta
ax.arrow(x, y, norte_x - x, norte_y - y, 
         color='white', head_width=30, head_length=25, label="Norte")
ax.arrow(x, y, este_x - x, este_y - y, 
         color='white', head_width=30, head_length=25, label="Este")

# Etiquetas
ax.text(norte_x + 25, norte_y - 50, 'N', color='white', fontsize=13, fontproperties=tnr_font, ha='center', va='bottom')
ax.text(este_x - 30, este_y + 10, 'E', color='white', fontsize=13, fontproperties=tnr_font, ha='center', va='bottom')


# Añadir la barra de color
cbar = plt.colorbar(im, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)
cbar.set_label('Brillo', fontproperties=tnr_font, fontsize=12)
cbar.ax.tick_params(labelsize=10)
for label in cbar.ax.get_yticklabels():
    label.set_fontproperties(tnr_font)


#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax.coords[0].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # α (RA)
ax.coords[1].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon = ax.coords[0]  # Eje de Ascensión Recta (α)
lat = ax.coords[1]  # Eje de Declinación (δ)


# Forzar la visualización completa de horas, minutos y segundos en α
lon.set_major_formatter('hh:mm:ss.s')

# Forzar la visualización completa de grados, minutos y segundos en δ
lat.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon.display_minor_ticks(True)
lat.display_minor_ticks(True)
#############################################################################


# Configurar la retícula correctamente
ax.grid(color='white', linestyle='--', linewidth=0.5)
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)  # Ajuste del espacio

plt.show()
F1000W.close()


### Base de código para realizar la composición RGB

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
import astropy.units as u
import matplotlib.font_manager as fm

# Fuente
font_path = "/usr/share/fonts/msttcore/times.ttf"
tnr_font = fm.FontProperties(fname=font_path)

# Cargar imágenes y WCS (asegurando abrir todos desde la misma carpeta)
F1130W_hdu = fits.open('NGC1300/NGC1300_JWST/jw02107-o002_t002_miri_f1130w_i2d.fits')  # Hα
F1000W_hdu = fits.open('NGC1300/NGC1300_JWST/jw02107-o002_t002_miri_f1000w_i2d.fits')  # [O III]
F770W_hdu = fits.open('NGC1300/NGC1300_JWST/jw02107-o002_t002_miri_f770w_i2d.fits')  # He II

F1130W = np.nan_to_num(F1130W_hdu[1].data)
F1000W = np.nan_to_num(F1000W_hdu[1].data)
F770W = np.nan_to_num(F770W_hdu[1].data)
wcs = WCS(F1130W_hdu[1].header)  # Usamos WCS de la primera imagen, por ejemplo

# Igualar tamaños
min_shape = (min(F1130W.shape[0], F1000W.shape[0], F770W.shape[0]), 
             min(F1130W.shape[1], F1000W.shape[1], F770W.shape[1]))

F1130W_crop = F1130W[:min_shape[0], :min_shape[1]]
F1000W_crop = F1000W[:min_shape[0], :min_shape[1]]
F770W_crop = F770W[:min_shape[0], :min_shape[1]]

# Normalización individual
def normalize_img(img):
    img_clip = np.clip(img, np.percentile(img[img > 0], 5), np.percentile(img[img > 0], 99))
    norm = (img_clip - img_clip.min()) / (img_clip.max() - img_clip.min())
    return norm

R = normalize_img(F1130W_crop)
G = normalize_img(F1000W_crop)
B = normalize_img(F770W_crop)

# Crear imagen RGB
rgb_image = np.zeros((R.shape[0], R.shape[1], 3))
rgb_image[:, :, 0] = R
rgb_image[:, :, 1] = G
rgb_image[:, :, 2] = B

# Función para flechas N y E
def N_E_arrows(ax, wcs):
    x, y = 1580, 150
    ra_dec_corner = wcs.pixel_to_world(x, y)
    delta_ra = 0.003 * u.deg
    delta_dec = 0.003 * u.deg
    ra_dec_norte = ra_dec_corner.spherical_offsets_by(0 * u.deg, delta_dec)
    ra_dec_este = ra_dec_corner.spherical_offsets_by(delta_ra, 0 * u.deg)
    norte_x, norte_y = wcs.world_to_pixel(ra_dec_norte)
    este_x, este_y = wcs.world_to_pixel(ra_dec_este)

    ax.arrow(x, y, norte_x - x, norte_y - y, color='gray', head_width=36, head_length=35)
    ax.arrow(x, y, este_x - x, este_y - y, color='gray', head_width=36, head_length=35)
    ax.text(norte_x, norte_y - 60, 'N', color='gray', fontsize=12, fontproperties=tnr_font, ha='center')
    ax.text(este_x - 35, este_y + 30, 'E', color='gray', fontsize=12, fontproperties=tnr_font, ha='center')

# Mostrar subplots con cada filtro y la combinación
fig = plt.figure(figsize=(20, 15))
plt.subplots_adjust(wspace=0.4)
# Hα
ax1 = fig.add_subplot(1, 4, 1, projection=wcs)
ax1.imshow(R, cmap='Reds')
ax1.set_title("F1130W ()", fontproperties=tnr_font, fontsize=20)
ax1.grid(color='gray', linestyle='--', linewidth=0.5)
N_E_arrows(ax1, wcs)
ax1.set_xlabel('δ', fontproperties=tnr_font, fontsize=12)
ax1.set_ylabel('α', fontproperties=tnr_font, fontsize=12)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax1.coords[0].set_ticklabel(size=18, fontproperties=tnr_font, simplify=False)  # α (RA)
ax1.coords[1].set_ticklabel(size=18, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon1 = ax1.coords[0]  # Eje de Ascensión Recta (α)
lat1 = ax1.coords[1]  # Eje de Declinación (δ)

# Forzar la visualización completa de horas, minutos y segundos en α
lon1.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat1.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon1.display_minor_ticks(True)
lat1.display_minor_ticks(True)
#############################################################################



# [O III]
ax2 = fig.add_subplot(1, 4, 2, projection=wcs)
ax2.imshow(G, cmap='Greens')
ax2.set_title("F1000W ()", fontproperties=tnr_font, fontsize=20)
ax2.grid(color='gray', linestyle='--', linewidth=0.5)
N_E_arrows(ax2, wcs)
ax2.set_xlabel('δ', fontproperties=tnr_font, fontsize=12)
ax2.set_ylabel('α', fontproperties=tnr_font, fontsize=12)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax2.coords[0].set_ticklabel(size=18, fontproperties=tnr_font, simplify=False)  # α (RA)
ax2.coords[1].set_ticklabel(size=18, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon2 = ax2.coords[0]  # Eje de Ascensión Recta (α)
lat2 = ax2.coords[1]  # Eje de Declinación (δ)

# Forzar la visualización completa de horas, minutos y segundos en α
lon2.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat2.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon2.display_minor_ticks(True)
lat2.display_minor_ticks(True)
#############################################################################


# He II
ax3 = fig.add_subplot(1, 4, 3, projection=wcs)
ax3.imshow(B, cmap='Blues')
ax3.set_title("F770W ()", fontproperties=tnr_font, fontsize=20)
ax3.grid(color='gray', linestyle='--', linewidth=0.5)
N_E_arrows(ax3, wcs)
ax3.set_xlabel('δ', fontproperties=tnr_font, fontsize=12)
ax3.set_ylabel('α', fontproperties=tnr_font, fontsize=12)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax3.coords[0].set_ticklabel(size=18, fontproperties=tnr_font, simplify=False)  # α (RA)
ax3.coords[1].set_ticklabel(size=18, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon3 = ax3.coords[0]  # Eje de Ascensión Recta (α)
lat3 = ax3.coords[1]  # Eje de Declinación (δ)


# Forzar la visualización completa de horas, minutos y segundos en α
lon3.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat3.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon3.display_minor_ticks(True)
lat3.display_minor_ticks(True)
#############################################################################


# RGB combinado
ax4 = fig.add_subplot(1, 4, 4, projection=wcs)
ax4.imshow(rgb_image)
ax4.set_title("NGC 1300 - Composición RGB", fontproperties=tnr_font, fontsize=20)
ax4.grid(color='white', linestyle='--', linewidth=0.5)
N_E_arrows(ax4, wcs)
ax4.set_xlabel('δ', fontproperties=tnr_font, fontsize=12)
ax4.set_ylabel('α', fontproperties=tnr_font, fontsize=12)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax4.coords[0].set_ticklabel(size=18, fontproperties=tnr_font, simplify=False)  # α (RA)
ax4.coords[1].set_ticklabel(size=18, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon4 = ax4.coords[0]  # Eje de Ascensión Recta (α)
lat4 = ax4.coords[1]  # Eje de Declinación (δ)

# Forzar la visualización completa de horas, minutos y segundos en α
lon4.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat4.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon4.display_minor_ticks(True)
lat4.display_minor_ticks(True)
#############################################################################


plt.tight_layout()
plt.show()


### Base de código para realizar los perfiles de brillo

In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

# Cargar los filtros RGB de la composición A
F1130W = np.nan_to_num(fits.getdata('NGC1300/NGC1300_JWST/jw02107-o002_t002_miri_f1130w_i2d.fits'))  # R
F1000W = np.nan_to_num(fits.getdata('NGC1300/NGC1300_JWST/jw02107-o002_t002_miri_f1000w_i2d.fits'))  # G
F770W = np.nan_to_num(fits.getdata('NGC1300/NGC1300_JWST/jw02107-o002_t002_miri_f770w_i2d.fits'))  # B

# Igualar tamaño
min_shape = (min(F1130W.shape[0], F1000W.shape[0], F770W.shape[0]),
             min(F1130W.shape[1], F1000W.shape[1], F770W.shape[1]))
F1130W = F1130W[:min_shape[0], :min_shape[1]]
F1000W = F1000W[:min_shape[0], :min_shape[1]]
F770W = F770W[:min_shape[0], :min_shape[1]]

# Centro
center_x, center_y = min_shape[0] // 2, min_shape[1] // 2

# Suavizado opcional (mejor para perfiles suaves)
F1130W_smooth = gaussian_filter(F1130W, sigma=1.5)
F1000W_smooth = gaussian_filter(F1000W, sigma=1.5)
F770W_smooth = gaussian_filter(F770W, sigma=1.5)

# Calcular perfil radial
def perfil_radial(imagen, centro, r_max):
    y, x = np.indices(imagen.shape)
    r = np.sqrt((x - centro[1])**2 + (y - centro[0])**2).astype(int)
    perfil = np.zeros(r_max)
    for i in range(r_max):
        mask = (r == i)
        if np.sum(mask) > 0:
            perfil[i] = np.mean(imagen[mask])
    return perfil

# Radios hasta los que analizar (ajusta si quieres más lejos)
r_max = 100

# Calcular perfiles para R, G, B
perfil_R = perfil_radial(F1130W_smooth, (center_x, center_y), r_max)
perfil_G = perfil_radial(F1000W_smooth, (center_x, center_y), r_max)
perfil_B = perfil_radial(F770W_smooth, (center_x, center_y), r_max)

# --- PLOT ---
plt.figure(figsize=(8,6))
plt.plot(perfil_R, color='red', label='(emisión térmica polvo intensa) - F1130W (R)')
plt.plot(perfil_G, color='green', label='(emisión ''equilibrada'') - F1000W (G)')
plt.plot(perfil_B, color='blue', label='(emisión térmica polvo menos intensa) - F770W (B)')
plt.title("Perfil Radial - Composición RGB (NGC 1300)")
plt.xlabel("Distancia radial (pixeles)")
plt.ylabel("Brillo promedio")
plt.legend(loc='upper right', bbox_to_anchor=(1, 0.85))
#plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## **NGC 5194 - Galaxia Remolino**

Base de código para el cálculo, en la imagen del HST, de: <br>
- centroide (MÁSCARA PARA NÚCLEO)
- diferenciación de las distintas regiones de la galaxia (SECCIONADO DE GALAXIA)
- transformación XY - V2V3 - NE (TRANSFORMACIÓN DE COORDENADAS)
- estrellas brillantes (DETECCIÓN DE ESTRELLAS BRILLANTES)
- distancia del centroide a las estrellas más brillants (DISTANCIAS FÍSICAS A ESTRELLAS BRILLANTES)
- visualización con todo lo anterior, orientación, retícula, ejes... (VISUALIZACIÓN)


In [ ]:
# Librerías necesarias
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
%matplotlib inline
from matplotlib.colors import LogNorm
from astropy.wcs import WCS
import math
from astropy import units as u
from astropy.coordinates import SkyCoord
from scipy.ndimage import label, center_of_mass
from astropy.visualization import AsinhStretch, MinMaxInterval, ImageNormalize
# Cargar la fuente directamente por su ruta
font_path = "/usr/share/fonts/msttcore/times.ttf"  
tnr_font = fm.FontProperties(fname=font_path)
from matplotlib.patches import Circle
from skimage.feature import peak_local_max
from sklearn.decomposition import PCA

###################### PASO DE AR - h m s Y DEC º ' " ######################
# Función para convertir grados a horas, minutos, segundos (1h son 15º)
def grados_a_hms(ar_deg):
    horas = int(ar_deg / 15)  # Convertir grados a horas
    minutos = int((ar_deg / 15 - horas) * 60)  # Convertir la parte fraccionaria de horas a minutos
    segundos = ((ar_deg / 15 - horas) * 60 - minutos) * 60  # Convertir la parte fraccionaria de minutos a segundos
    return f"{horas}h {minutos}m {segundos:.2f}s"
# Convertir AR a formato de horas, minutos y segundos
#AR_hms = grados_a_hms(AR)

# Función para convertir grados a grados, minutos y segundos
def grados_a_gms(deg):
    grados = int(deg)  # Parte entera de los grados
    minutos = int(abs(deg - grados) * 60)  # Convertir la parte decimal de grados a minutos
    segundos = (abs(deg - grados) * 60 - minutos) * 60  # Convertir la parte decimal de minutos a segundos
    return f"{grados}° {minutos}' {segundos:.2f}'' "

# Convertir DEC a formato de grados, minutos y segundos
#DEC_gms = grados_a_gms(DEC)
###################################################################################

NGC5194 = fits.open('NGC5194/MAST_2024-10-14T1013/HST/jd8f21010/jd8f21010_drz.fits')
M51 = fits.getdata('NGC5194/MAST_2024-10-14T1013/HST/jd8f21010/jd8f21010_drz.fits')
# Reemplazar NaN por 0
M51 = np.nan_to_num(M51)

# Obtener las dimensiones de la imagen
height, width = M51.shape
print('Píxeles de la imagen:',M51.shape)

# Calcular el centro de la imagen
center_x, center_y = height // 2, width // 2
print('El centro de la imagen es: (' ,center_x,',', center_y, ')')

# Obtener WCS
wcs = WCS(NGC5194[1].header, naxis=2)

#####################################   MÁSCARA PARA NÚCLEO   ####################################
# Paso 1: Crear una máscara para la región más brillante
brightness_threshold = 0.30 * np.max(M51)  # Definir umbral (10% del brillo máximo)
mask = M51 > brightness_threshold  # Máscara de los píxeles más brillantes

# Paso 2: Encontrar las regiones conectadas y seleccionar la más grande (núcleo)
labeled, num_features = label(mask)
region_sizes = np.array([(labeled == i).sum() for i in range(1, num_features + 1)])
largest_region_idx = np.argmax(region_sizes) + 1  # Índice de la región más grande
mask = labeled == largest_region_idx  # Filtrar solo la región más grande

# Paso 3: Encontrar el centroide ponderado de la región filtrada
centroid = center_of_mass(M51 * mask)
centroid = (int(centroid[0]), int(centroid[1]))  # Convertir a enteros
print(f"Centro refinado de M51: {centroid}")

# Paso 4: Convertir a coordenadas ecuatoriales (RA, DEC)
RA_centroid, DEC_centroid = wcs.all_pix2world([[centroid[1], centroid[0]]], 1)[0]

# Convertir a formato legible
RA_centroid_hms = grados_a_hms(RA_centroid)
DEC_centroid_gms = grados_a_gms(DEC_centroid)
# Mostrar resultados
print('')
print(f"Centro refinado en coordenadas celestes (RA, DEC):")
print(f" - RA: {RA_centroid:.6f}")
print(f" - DEC: {DEC_centroid:.6f}")
print('')
print(f"Centro refinado en coordenadas celestes (RA, DEC):")
print(f" - RA: {RA_centroid_hms}")
print(f" - DEC: {DEC_centroid_gms}")


#####################################   SECCIONADO DE GALAXIA   #######################################
# Calcular vmin y vmax globales para LogNorm
vmin = np.percentile(M51[M51 > 0], 5)   # 5% más oscuro
vmax = np.percentile(M51[M51 > 0], 99.5) # 99.5% más brillante

# --- Suavizar imagen para destacar regiones brillantes ---
M51_smooth = gaussian_filter(M51, sigma=5)  # Ajusta sigma según nivel de suavizado deseado

# --- Definir niveles de contorno ---
levels = np.logspace(np.log10(vmin), np.log10(vmax), num=5)  # Número de contornos

#####################################   DETECCIÓN DE ESTRELLAS BRILLANTES   #####################################
# Paso 1: Normalización (si no la tienes ya hecha)
img_norm = (M51 - np.min(M51)) / (np.max(M51) - np.min(M51))

# Paso 2: Detección de picos locales (estrellas brillantes)
coords_estrellas = peak_local_max(img_norm, min_distance=5, threshold_abs=0.975)  # Puedes ajustar el umbral

# Mensaje informativo
print(f"Se han detectado y marcado las {len(coords_estrellas)} estrellas más brillantes.")

####################################   DISTANCIAS FÍSICAS A ESTRELLAS BRILLANTES  #####################################

# Distancia en años luz
d_ly = 31000000 # información sacada de internet
ly_to_pc = 0.3066
d_pc = d_ly * ly_to_pc  

# Escala de píxel (arcsec/pix)
pixel_scales = np.abs(wcs.pixel_scale_matrix.diagonal()) * 3600
arcsec_per_pix = np.mean(pixel_scales)

# Calcular distancias en píxeles desde el centroide
pix_distances = [np.linalg.norm(np.array([y, x]) - np.array(centroid)) for y, x in coords_estrellas]

# Pasar a arcsec
arcsec_distances = np.array(pix_distances) * arcsec_per_pix

# Pasar a radianes
theta_rad = (arcsec_distances / 3600) * (np.pi / 180)

# Distancias físicas en parsec
pc_distances = theta_rad * d_pc

# Mostrar 5 resultados más representativos (puedes ajustar el número)
print('\nDistancias desde el núcleo a las estrellas más brillantes:')
for i, (p, a, pc) in enumerate(zip(pix_distances[:6], arcsec_distances[:6], pc_distances[:6]), 1):
    print(f"Estrella {i}: {p:.1f} px ≈ {a:.2f}\" ≈ {pc:.2f} pc")

#####################################   VISUALIZACIÓN   #####################################
# 📌 --- 1. Ajustar la Normalización ---
#vmin = np.percentile(M51[M51 > 0], 5)   # 5% más oscuro
#vmax = np.percentile(M51[M51 > 0], 99.5) # 99.5% más brillante
# Normalización avanzada usando AsinhStretch
norm = ImageNormalize(M51, interval=MinMaxInterval(), stretch=AsinhStretch())

# Crear la figura y el eje
fig = plt.figure(figsize=(9, 9))
ax = plt.subplot(projection=wcs)  ## Trabajar en 2D
# Mostrar la imagen de M51 con el colormap y escala logarítmica
im = ax.imshow(M51, cmap='magma', norm=LogNorm(vmin=vmin, vmax=vmax)) 
# --- Dibujar contornos que resalten las regiones ---
ax.contour(M51_smooth, levels=levels, colors='cyan', linewidths=0.9, alpha=0.5)

ax.set_title('NGC 5194 - Galaxia Remolino (F606W)', fontproperties=tnr_font, fontsize=16, pad =15)
#ax.set_title('NGC 5194 - Galaxia Remolino', fontproperties=tnr_font, fontsize=16, pad =15)
ax.set_xlabel('α', fontproperties=tnr_font, fontsize=12)
ax.set_ylabel('δ', fontproperties=tnr_font, fontsize=12)

# Marcar el centro estimado en la imagen
ax.plot(centroid[1], centroid[0], '+', color='black', markersize=8)

# Dibujar círculos estrellas más brillantes y líneas desde el núcleo
for i, (y, x) in enumerate(coords_estrellas[:6]):  # solo las 5 más cercanas por claridad
    ax.add_patch(Circle((x, y), radius=10, edgecolor='lime', facecolor='none', linewidth=1.6))
    ax.plot([centroid[1], x], [centroid[0], y], color='orange', linestyle='--', linewidth=1.2)
    dy = - 12 if i % 2 == 0 else 140  # alterna arriba y abajo
    dx = 12
    ax.text(x + dx, y + dy, f'{i+1}', color='white', fontsize=13, fontproperties=tnr_font,
            ha='left', va='center', bbox=dict(facecolor='black', alpha=0.5, pad=1.5))

# --------------------------
# Añadir Norte y Este en la retícula correcta
# Punto en la esquina inferior derecha para la orientación
x, y = 3383.0, 527.25
ra_dec_corner = wcs.pixel_to_world(x, y)

# Desplazamientos para las flechas
delta_ra = (0.004 * u.deg)
delta_dec = (0.004 * u.deg)

# Calcular coordenadas de Norte y Este
ra_dec_norte = ra_dec_corner.spherical_offsets_by(0 * u.deg, delta_dec)
ra_dec_este = ra_dec_corner.spherical_offsets_by(delta_ra, 0 * u.deg)

# Convertir de vuelta a píxeles
norte_x, norte_y = wcs.world_to_pixel(ra_dec_norte)
este_x, este_y = wcs.world_to_pixel(ra_dec_este)

# Dibujar flechas de Norte y Este en la esquina inferior derecha
ax.arrow(x, y, norte_x - x, norte_y - y, 
         color='white', head_width=35, head_length=33, label="Norte")
ax.arrow(x, y, este_x - x, este_y - y, 
         color='white', head_width=35, head_length=33, label="Este")

# Etiquetas en la esquina inferior derecha
ax.text(norte_x - 90, norte_y - 45, 'N', color='white', fontsize=13, fontproperties=tnr_font, ha='center', va='bottom')
ax.text(este_x, este_y + 20, 'E', color='white', fontsize=13, fontproperties=tnr_font, ha='center', va='bottom')


# Añadir la barra de color
cbar = fig.colorbar(im, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)
cbar.set_label('Brillo', fontproperties=tnr_font, fontsize=12)
cbar.ax.tick_params(labelsize=10)
for label in cbar.ax.get_yticklabels():
    label.set_fontproperties(tnr_font)

# Añadir anotación en la imagen
text_str = f"Coordenadas del núcleo\nα: {RA_centroid_hms}\nδ: {DEC_centroid_gms}"
ax.text(0.95, 0.95, text_str, transform=ax.transAxes, color='white', fontsize=10,
        fontweight='bold', fontproperties=tnr_font, ha='right', va='top', bbox=dict(facecolor='black', alpha=0.5))

#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax.coords[0].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # α (RA)
ax.coords[1].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon = ax.coords[0]  # Eje de Ascensión Recta (α)
lat = ax.coords[1]  # Eje de Declinación (δ)


# Forzar la visualización completa de horas, minutos y segundos en α
lon.set_major_formatter('hh:mm:ss.s')

# Forzar la visualización completa de grados, minutos y segundos en δ
lat.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon.display_minor_ticks(True)
lat.display_minor_ticks(True)
#############################################################################
# Añadir una retícula
ax.grid(color='black', linestyle='--', linewidth=0.5)


plt.show()
NGC5194.close()

Base de código para el cálculo, en la imagen del JWST, de: <br>
- centroide (MÁSCARA PARA NÚCLEO)
- diferenciación de las distintas regiones de la galaxia (SECCIONADO DE GALAXIA)
- transformación XY - V2V3 - NE (TRANSFORMACIÓN DE COORDENADAS)
- estrellas brillantes (DETECCIÓN DE ESTRELLAS BRILLANTES)
- distancia del centroide a las estrellas más brillants (DISTANCIAS FÍSICAS A ESTRELLAS BRILLANTES)
- visualización con todo lo anterior, orientación, retícula, ejes... (VISUALIZACIÓN)


In [ ]:
# Librerías necesarias
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib.colors import LogNorm
from astropy.wcs import WCS
import math
from astropy import units as u
from astropy.coordinates import SkyCoord
from scipy.ndimage import label, center_of_mass
from skimage.measure import regionprops
from matplotlib.patches import Circle
from skimage.feature import peak_local_max
from sklearn.decomposition import PCA

# Cargar la fuente
font_path = "/usr/share/fonts/msttcore/times.ttf"
tnr_font = fm.FontProperties(fname=font_path)

# Cargar los datos de la imagen
F150W = fits.open('NGC5194/M51_JWST/jw01783-o003_t009_nircam_clear-f150w_i2d.fits')  # FITS file
F150W_h = F150W[0].header
F150W_d = fits.getdata('NGC5194/M51_JWST/jw01783-o003_t009_nircam_clear-f150w_i2d.fits')
F150W_d = np.nan_to_num(F150W_d)  # Reemplazar NaN por 0

# Obtener dimensiones
height, width = F150W_d.shape
print(f"Píxeles de la imagen: {F150W_d.shape}")
print('----------------------------------------------------------------------------')
# Obtener WCS
wcs = WCS(F150W[1].header, naxis=2)

###################### PASO DE AR - h m s Y DEC º ' " ######################
# Función para convertir grados a horas, minutos, segundos (1h son 15º)
def grados_a_hms(ar_deg):
    horas = int(ar_deg / 15)  # Convertir grados a horas
    minutos = int((ar_deg / 15 - horas) * 60)  # Convertir la parte fraccionaria de horas a minutos
    segundos = ((ar_deg / 15 - horas) * 60 - minutos) * 60  # Convertir la parte fraccionaria de minutos a segundos
    return f"{horas}h {minutos}m {segundos:.2f}s"
# Convertir AR a formato de horas, minutos y segundos
#AR_hms = grados_a_hms(AR)

# Función para convertir grados a grados, minutos y segundos
def grados_a_gms(deg):
    grados = int(deg)  # Parte entera de los grados
    minutos = int(abs(deg - grados) * 60)  # Convertir la parte decimal de grados a minutos
    segundos = (abs(deg - grados) * 60 - minutos) * 60  # Convertir la parte decimal de minutos a segundos
    return f"{grados}° {minutos}' {segundos:.2f}'' "

# Convertir DEC a formato de grados, minutos y segundos
#DEC_gms = grados_a_gms(DEC)
####################################################

#####################################   MÁSCARA PARA NÚCLEO    ##################################################
# Definir el radio de interés en píxeles para el núcleo
R_nucleo = 6500  # Radio en píxeles

# Obtener el centro de la imagen
center_x, center_y = height // 2, width // 2

# Crear una máscara circular para limitar la búsqueda del núcleo
Y, X = np.ogrid[:height, :width]
mask_circle_nucleo = (X - center_x) ** 2 + (Y - center_y) ** 2 <= R_nucleo ** 2  # Ecuación del círculo

# Aplicar la máscara circular a la imagen original
masked_F150W_nucleo = np.where(mask_circle_nucleo, F150W_d, 0)

# Aplicar umbral de brillo SOLO dentro del círculo del núcleo
umbral_nucleo = np.percentile(masked_F150W_nucleo[mask_circle_nucleo], 98)
mask_nucleo = masked_F150W_nucleo > umbral_nucleo

# Encontrar las regiones conectadas del núcleo
labeled, num_features = label(mask_nucleo)
region_sizes = np.bincount(labeled.ravel())[1:]  # Contar tamaños de regiones
largest_region_idx = np.argmax(region_sizes) + 1  # Índice de la región más grande
mask_nucleo = labeled == largest_region_idx  # Filtrar solo la región central

# Obtener el centroide ponderado (núcleo de la nebulosa)
centroid = center_of_mass(masked_F150W_nucleo * mask_nucleo)
centroid = (int(centroid[0]), int(centroid[1]))  # Convertir a enteros
print(f"Centro refinado de NGC 5194: {centroid}")

# Convertir a coordenadas celestes
RA_centroid, DEC_centroid = wcs.all_pix2world([[centroid[1], centroid[0]]], 1)[0]
RA_centroid_hms = grados_a_hms(RA_centroid)
DEC_centroid_gms = grados_a_gms(DEC_centroid)
print(f"Centro refinado de NGC 5194 en coordendas (α,δ): ({RA_centroid_hms}, {DEC_centroid_gms}) ")
print('----------------------------------------------------------------------------')

#####################################   TRANSFORMACIÓN DE COORDENADAS   ##################################
# Calcular escala de píxel (en grados/píxel → luego se convierte a arcsec/píxel)
pixel_scales_deg = wcs.pixel_scale_matrix.diagonal()
pixel_scale = np.abs(pixel_scales_deg) * 3600  # Conversión a arcsec

# Datos de entrada
# Sabiendo que APERNAME= 'NRCA1_FULL' usamos los valores de A1
Xr = -122.49
Yr = -34.77
V2r = 120.58  # en arcosegundos
V3r = -527.50 # en arcosegundos

Sx, Sy = pixel_scale[0], pixel_scale[1]

beta_x = 0  # en grados
beta_y = 90  # en grados
 # Centroide de la imagen, punto a transformar
X = 2774.0
Y = 5439.0
PA_V3 = 130.7514872273834  # Ángulo PA_V3 en grados

# Conversión de grados a radianes, para poder trabajar con ellos
beta_x_rad = math.radians(beta_x)
beta_y_rad = math.radians(beta_y)
# Convertir PA_V3 a radianes
PA_V3_rad = math.radians(PA_V3)

# Cálculo de V2 arcosegundos
V2 = V2r + Sx * math.sin(beta_x_rad) * (X - Xr) + Sy * math.sin(beta_y_rad) * (Y - Yr)

# Cálculo de V3 arcosegundos
V3 = V3r + Sx * math.cos(beta_x_rad) * (X - Xr) + Sy * math.cos(beta_y_rad) * (Y - Yr)

# Rotación de las coordenadas V2 y V3 al sistema Norte y Este
N = V3 * math.cos(PA_V3_rad) + V2 * math.sin(PA_V3_rad)  # Coordenada Norte relativa
E = V3 * math.sin(PA_V3_rad) + V2 * math.cos(PA_V3_rad)  # Coordenada Este relativa


# Conversión a arcmin (1 arcmin = 60 arcsec)
N_arcmin = N / 60
E_arcmin = E / 60

# Mostrar los resultados
print(f"Coordenada Norte: {N:.2f} arcsec → {N_arcmin:.2f} arcmin")
print(f"Coordenada Este : {E:.2f} arcsec → {E_arcmin:.2f} arcmin")
print('')
#####################################   SECCIONADO DE GALAXIA   #######################################
# Calcular vmin y vmax globales para LogNorm
vmin = np.percentile(F150W_d[F150W_d > 0], 5)   # evitar valores 0
vmax = np.percentile(F150W_d[F150W_d > 0], 99) # 99% más brillante 

# --- Suavizar imagen para destacar regiones brillantes ---
F150W_smooth = gaussian_filter(F150W_d, sigma=6)  # Ajusta sigma según nivel de suavizado deseado

# --- Definir niveles de contorno ---
levels = np.logspace(np.log10(vmin), np.log10(vmax), num=4)  # Número de contornos

#####################################   DETECCIÓN DE ESTRELLAS BRILLANTES   #####################################
# Paso 1: Normalización (si no la tienes ya hecha)
img_norm = (F150W_d - np.min(F150W_d)) / (np.max(F150W_d) - np.min(F150W_d))

# Paso 2: Detección de picos locales (estrellas brillantes)
coords_estrellas = peak_local_max(img_norm, min_distance=5, threshold_abs=0.75)  # Puedes ajustar el umbral

# Mensaje informativo
print(f"Se han detectado y marcado las {len(coords_estrellas)} estrellas más brillantes.")

####################################   DISTANCIAS FÍSICAS A ESTRELLAS BRILLANTES  #####################################

# Distancia en años luz
d_ly = 31000000 # información sacada de internet
ly_to_pc = 0.3066
d_pc = d_ly * ly_to_pc  

# Escala de píxel (arcsec/pix)
pixel_scales = np.abs(wcs.pixel_scale_matrix.diagonal()) * 3600
arcsec_per_pix = np.mean(pixel_scales)

# Calcular distancias en píxeles desde el centroide
pix_distances = [np.linalg.norm(np.array([y, x]) - np.array(centroid)) for y, x in coords_estrellas]

# Pasar a arcsec
arcsec_distances = np.array(pix_distances) * arcsec_per_pix

# Pasar a radianes
theta_rad = (arcsec_distances / 3600) * (np.pi / 180)

# Distancias físicas en parsec
pc_distances = theta_rad * d_pc

# Mostrar 5 resultados más representativos (puedes ajustar el número)
print('\nDistancias desde el núcleo a las estrellas más brillantes:')
for i, (p, a, pc) in enumerate(zip(pix_distances[:7], arcsec_distances[:7], pc_distances[:7]), 1):
    print(f"Estrella {i}: {p:.1f} px ≈ {a:.2f}\" ≈ {pc:.2f} pc")

#####################################   VISUALIZACIÓN   #######################################

# Crear la figura sin distorsiones
fig = plt.figure(figsize=(18, 10))
ax = plt.subplot(projection=wcs)  ## Trabajar en 2D

# Calcular vmin y vmax globales para LogNorm
#vmin = np.percentile(F150W_d[F150W_d > 0], 5)   # evitar valores 0
#vmax = np.percentile(F150W_d[F150W_d > 0], 99) # 99% más brillante 

# Mostrar la imagen con mejor contraste
im = ax.imshow(F150W_d, cmap='inferno', norm=LogNorm(vmin=vmin, vmax=vmax))
ax.set_title('NGC 5194 - Galaxia Remolino (F150W)', fontproperties=tnr_font, fontsize=16, pad=40)
# --- Dibujar contornos que resalten las regiones ---
ax.contour(F150W_smooth, levels=levels, colors='cyan', linewidths=0.9, alpha=0.5)
ax.set_xlabel('δ', fontproperties=tnr_font, fontsize=12)
ax.set_ylabel('α', fontproperties=tnr_font, fontsize=12)
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax.coords[0].set_ticklabel(size=12, fontproperties=tnr_font)  # RA (α)
ax.coords[1].set_ticklabel(size=12, fontproperties=tnr_font)  # DEC (δ)


# Marcar el centro refinado en la imagen
ax.plot(centroid[1], centroid[0], '+', color='black', markersize=8, label="Centro Refinado")

# Dibujar círculos estrellas más brillantes y líneas desde el núcleo
for i, (y, x) in enumerate(coords_estrellas[:7]):  # solo las 5 más cercanas por claridad
    ax.add_patch(Circle((x, y), radius=10, edgecolor='lime', facecolor='none', linewidth=1.6))
    ax.plot([centroid[1], x], [centroid[0], y], color='orange', linestyle='--', linewidth=1.2)
    ax.text(x - 130, y, f'{i+1}', color='white', fontsize=13, fontproperties=tnr_font,
            ha='left', va='center', bbox=dict(facecolor='black', alpha=0.5, pad=1.5))

# Anotación con coordenadas
text_str = f"Coordenadas del núcleo\nα: {grados_a_hms(RA_centroid)}\nδ: {grados_a_gms(DEC_centroid)}"
ax.text(0.93, 0.95, text_str, transform=ax.transAxes, color='white', fontsize=12,
        fontweight='bold', fontproperties=tnr_font, ha='right', va='top', bbox=dict(facecolor='black', alpha=0.5))

# Añadir leyenda para N y E
#text_str = f"Nc: {N:.6f} (arcsec)\nEc: {E:.6f} (arcsec)"
#ax.text(0.95, 0.84, text_str, transform=ax.transAxes, color='white', fontsize=10,
#       fontweight='bold', fontproperties=tnr_font, ha='right', va='top', bbox=dict(facecolor='black', alpha=0.5))

# --------------------------
# Añadir Norte y Este en la retícula correcta
# Punto en la esquina inferior derecha para la orientación
x, y = 10350.0, 1000.2
ra_dec_corner = wcs.pixel_to_world(x, y)

# Desplazamientos para las flechas
delta_ra = (0.004 * u.deg)
delta_dec = (0.004 * u.deg)

# Calcular coordenadas de Norte y Este
ra_dec_norte = ra_dec_corner.spherical_offsets_by(0 * u.deg, delta_dec)
ra_dec_este = ra_dec_corner.spherical_offsets_by(delta_ra, 0 * u.deg)

# Convertir de vuelta a píxeles
norte_x, norte_y = wcs.world_to_pixel(ra_dec_norte)
este_x, este_y = wcs.world_to_pixel(ra_dec_este)

# Dibujar flechas en la dirección correcta
ax.arrow(x, y, norte_x - x, norte_y - y, 
         color='white', head_width=70, head_length=80, label="Norte")
ax.arrow(x, y, este_x - x, este_y - y, 
         color='white', head_width=70, head_length=80, label="Este")

# Etiquetas
ax.text(norte_x - 90, norte_y - 150, 'N', color='white', fontsize=13, fontproperties=tnr_font, ha='center', va='bottom')
ax.text(este_x - 60, este_y + 75, 'E', color='white', fontsize=13, fontproperties=tnr_font, ha='center', va='bottom')

# Añadir la barra de color
cbar = plt.colorbar(im, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)
cbar.set_label('Brillo', fontproperties=tnr_font, fontsize=12)
cbar.ax.tick_params(labelsize=10)
for label in cbar.ax.get_yticklabels():
    label.set_fontproperties(tnr_font)

#############################################################################
# Modificar las etiquetas y formatos de los ejes (α y δ)
ax.coords[0].set_axislabel('α', fontsize=12, fontproperties=tnr_font)  # Eje horizontal
ax.coords[1].set_axislabel('δ', fontsize=12, fontproperties=tnr_font)  # Eje vertical

# Estilo de las etiquetas
ax.coords[0].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)
ax.coords[1].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)

# Formato de unidades
ax.coords[0].set_major_formatter('hh:mm:ss.s')  # Ascensión recta
ax.coords[1].set_major_formatter('dd:mm:ss')    # Declinación

# Ticks menores
ax.coords[0].display_minor_ticks(True)
ax.coords[1].display_minor_ticks(True)

# Malla de retícula
ax.coords.grid(True, color='white', linestyle='--', linewidth=0.5)
ax.coords[0].set_ticks(number=10)
ax.coords[1].set_ticks(number=10)

#############################################################################


# Configurar la retícula correctamente
#ax.grid(color='white', linestyle='--', linewidth=0.5)
ax.coords.grid(True, color='white', linestyle='--', linewidth=0.5)
ax.coords[0].set_ticks(number=7)
ax.coords[1].set_ticks(number=7)

plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)  # Ajuste del espacio

plt.show()
F150W.close()


### Base de código para realizar la composición RGB

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
import astropy.units as u
import matplotlib.font_manager as fm

# Fuente
font_path = "/usr/share/fonts/msttcore/times.ttf"
tnr_font = fm.FontProperties(fname=font_path)

# Cargar imágenes y WCS (asegurando abrir todos desde la misma carpeta)
F187N_hdu = fits.open('NGC5194/M51_JWST/jw01783-o003_t009_nircam_clear-f187n_i2d.fits')  # Hα
F150W_hdu = fits.open('NGC5194/M51_JWST/jw01783-o003_t009_nircam_clear-f150w_i2d.fits')  # [O III]
F115W_hdu = fits.open('NGC5194/M51_JWST/jw01783-o003_t009_nircam_clear-f115w_i2d.fits')  # He II

F187N = np.nan_to_num(F187N_hdu[1].data)
F150W = np.nan_to_num(F150W_hdu[1].data)
F115W = np.nan_to_num(F115W_hdu[1].data)
# wcs = WCS(F200W_hdu[1].header)   Usamos WCS de la primera imagen, por ejemplo
wcs = WCS(F187N_hdu[1].header, naxis=2)



# Igualar tamaños
min_shape = (min(F187N.shape[0], F150W.shape[0], F115W.shape[0]), 
             min(F187N.shape[1], F150W.shape[1], F115W.shape[1]))

F187N_crop = F187N[:min_shape[0], :min_shape[1]]
F150W_crop = F150W[:min_shape[0], :min_shape[1]]
F115W_crop = F115W[:min_shape[0], :min_shape[1]]

# Normalización individual
def normalize_img(img):
    img_clip = np.clip(img, np.percentile(img[img > 0], 5), np.percentile(img[img > 0], 99))
    norm = (img_clip - img_clip.min()) / (img_clip.max() - img_clip.min())
    return norm

R = normalize_img(F187N_crop)
G = normalize_img(F150W_crop)
B = normalize_img(F115W_crop)

# Crear imagen RGB
rgb_image = np.zeros((R.shape[0], R.shape[1], 3))
rgb_image[:, :, 0] = R
rgb_image[:, :, 1] = G
rgb_image[:, :, 2] = B

# Función para flechas N y E
def N_E_arrows(ax, wcs):
    x, y = 9950, 600
    ra_dec_corner = wcs.pixel_to_world(x, y)
    delta_ra = 0.003 * u.deg
    delta_dec = 0.003 * u.deg
    ra_dec_norte = ra_dec_corner.spherical_offsets_by(0 * u.deg, delta_dec)
    ra_dec_este = ra_dec_corner.spherical_offsets_by(delta_ra, 0 * u.deg)
    norte_x, norte_y = wcs.world_to_pixel(ra_dec_norte)
    este_x, este_y = wcs.world_to_pixel(ra_dec_este)

    ax.arrow(x, y, norte_x - x, norte_y - y, color='gray', head_width=40, head_length=35)
    ax.arrow(x, y, este_x - x, este_y - y, color='gray', head_width=40, head_length=35)
    ax.text(norte_x - 65, norte_y - 155, 'N', color='gray', fontsize=12, fontproperties=tnr_font, ha='center')
    ax.text(este_x - 25, este_y + 50, 'E', color='gray', fontsize=12, fontproperties=tnr_font, ha='center')

# Mostrar subplots con cada filtro y la combinación
fig = plt.figure(figsize=(22, 16))

# Hα
ax1 = fig.add_subplot(2, 3, 1, projection=wcs)
ax1.imshow(R, cmap='Reds')
ax1.set_title("F187N ()", fontproperties=tnr_font, fontsize=13, pad=40)
#ax1.grid(color='gray', linestyle='--', linewidth=0.5)
ax1.coords.grid(True, color='gray', linestyle='--', linewidth=0.7)
ax1.coords[0].set_ticks(number=7)
ax1.coords[1].set_ticks(number=7)

N_E_arrows(ax1, wcs)
ax1.coords[0].set_axislabel('α', fontproperties=tnr_font, fontsize=12)
ax1.coords[1].set_axislabel('δ', fontproperties=tnr_font, fontsize=12)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax1.coords[0].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # α (RA)
ax1.coords[1].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon1 = ax1.coords[0]  # Eje de Ascensión Recta (α)
lat1 = ax1.coords[1]  # Eje de Declinación (δ)

# Forzar la visualización completa de horas, minutos y segundos en α
lon1.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat1.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon1.display_minor_ticks(True)
lat1.display_minor_ticks(True)
#############################################################################



# [O III]
ax2 = fig.add_subplot(2, 3, 2, projection=wcs)
ax2.imshow(G, cmap='Greens')
ax2.set_title("F150W ()", fontproperties=tnr_font, fontsize=13, pad=40)
#ax2.grid(color='gray', linestyle='--', linewidth=0.5)
ax2.coords.grid(True, color='gray', linestyle='--', linewidth=0.7)
ax2.coords[0].set_ticks(number=7)
ax2.coords[1].set_ticks(number=7)

N_E_arrows(ax2, wcs)
ax2.coords[0].set_axislabel('α', fontproperties=tnr_font, fontsize=12)
ax2.coords[1].set_axislabel('δ', fontproperties=tnr_font, fontsize=12)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax2.coords[0].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # α (RA)
ax2.coords[1].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon2 = ax2.coords[0]  # Eje de Ascensión Recta (α)
lat2 = ax2.coords[1]  # Eje de Declinación (δ)

# Forzar la visualización completa de horas, minutos y segundos en α
lon2.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat2.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon2.display_minor_ticks(True)
lat2.display_minor_ticks(True)
#############################################################################


# He II
ax3 = fig.add_subplot(2, 3, 3, projection=wcs)
ax3.imshow(B, cmap='Blues')
ax3.set_title("F115W ()", fontproperties=tnr_font, fontsize=13, pad=40)
#ax3.grid(color='gray', linestyle='--', linewidth=0.5)
ax3.coords.grid(True, color='gray', linestyle='--', linewidth=0.7)
ax3.coords[0].set_ticks(number=7)
ax3.coords[1].set_ticks(number=7)

N_E_arrows(ax3, wcs)
ax3.coords[0].set_axislabel('α', fontproperties=tnr_font, fontsize=12)
ax3.coords[1].set_axislabel('δ', fontproperties=tnr_font, fontsize=12)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax3.coords[0].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # α (RA)
ax3.coords[1].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon3 = ax3.coords[0]  # Eje de Ascensión Recta (α)
lat3 = ax3.coords[1]  # Eje de Declinación (δ)


# Forzar la visualización completa de horas, minutos y segundos en α
lon3.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat3.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon3.display_minor_ticks(True)
lat3.display_minor_ticks(True)
#############################################################################


# RGB combinado
ax4 = fig.add_subplot(2, 3, 5, projection=wcs)
ax4.imshow(rgb_image)
ax4.set_title("NGC 5194 - Composición RGB (A)", fontproperties=tnr_font, fontsize=13, pad=40)
#ax4.grid(color='white', linestyle='--', linewidth=0.5)
ax4.coords.grid(True, color='gray', linestyle='--', linewidth=0.7)
ax4.coords[0].set_ticks(number=7)
ax4.coords[1].set_ticks(number=7)

N_E_arrows(ax4, wcs)
ax4.coords[0].set_axislabel('α', fontproperties=tnr_font, fontsize=12)
ax4.coords[1].set_axislabel('δ', fontproperties=tnr_font, fontsize=12)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax4.coords[0].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # α (RA)
ax4.coords[1].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon4 = ax4.coords[0]  # Eje de Ascensión Recta (α)
lat4 = ax4.coords[1]  # Eje de Declinación (δ)

# Forzar la visualización completa de horas, minutos y segundos en α
lon4.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat4.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon4.display_minor_ticks(True)
lat4.display_minor_ticks(True)
#############################################################################



plt.tight_layout()
plt.show()


### Base de código para realizar los perfiles de brillo

In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

# Cargar los filtros RGB de la composición A
F187N = np.nan_to_num(fits.getdata('NGC5194/M51_JWST/jw01783-o003_t009_nircam_clear-f187n_i2d.fits'))  # R
F150W = np.nan_to_num(fits.getdata('NGC5194/M51_JWST/jw01783-o003_t009_nircam_clear-f150w_i2d.fits'))  # G
F115W = np.nan_to_num(fits.getdata('NGC5194/M51_JWST/jw01783-o003_t009_nircam_clear-f115w_i2d.fits'))  # B

# Igualar tamaño
min_shape = (min(F187N.shape[0], F150W.shape[0], F115W.shape[0]),
             min(F187N.shape[1], F150W.shape[1], F115W.shape[1]))
F187N = F187N[:min_shape[0], :min_shape[1]]
F150W = F150W[:min_shape[0], :min_shape[1]]
F115W = F115W[:min_shape[0], :min_shape[1]]

# Centro
center_x, center_y = min_shape[0] // 2, min_shape[1] // 2

# Suavizado opcional (mejor para perfiles suaves)
F187N_smooth = gaussian_filter(F187N, sigma=1.5)
F150W_smooth = gaussian_filter(F150W, sigma=1.5)
F115W_smooth = gaussian_filter(F115W, sigma=1.5)

# Calcular perfil radial
def perfil_radial(imagen, centro, r_max):
    y, x = np.indices(imagen.shape)
    r = np.sqrt((x - centro[1])**2 + (y - centro[0])**2).astype(int)
    perfil = np.zeros(r_max)
    for i in range(r_max):
        mask = (r == i)
        if np.sum(mask) > 0:
            perfil[i] = np.mean(imagen[mask])
    return perfil

# Radios hasta los que analizar (ajusta si quieres más lejos)
r_max = 100

# Calcular perfiles para R, G, B
perfil_R = perfil_radial(F187N_smooth, (center_x, center_y), r_max)
perfil_G = perfil_radial(F150W_smooth, (center_x, center_y), r_max)
perfil_B = perfil_radial(F115W_smooth, (center_x, center_y), r_max)

# --- PLOT ---
plt.figure(figsize=(8,6))
plt.plot(perfil_R, color='red', label='[H II] - F187N (R)')
plt.plot(perfil_G, color='green', label='(emisión ''equilibrada'' en el NIR) - F150W (G)')
plt.plot(perfil_B, color='blue', label='(emisiones centradas en NIR) - F115W (B)')
plt.title("Perfil Radial - Composición RGB A (NGC 5194)")
plt.xlabel("Distancia radial (pixeles)")
plt.ylabel("Brillo promedio")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## **NGC 4038 - Sistema galáctico de Las Antenas**

Base de código para el cálculo de: <br>
- centroides de NGC 4038 (MÁSCARA PARA NÚCLEO)
- centroide de NGC 4039 (NÚCLEOS DEL SISTEMA)
- distancia entre ambos núcleos (DISTANCIA ENTRE NÚCLEOS SABIENDO LA DISTANCIA AL SISTEMA GALÁCTICO)
- transformación XY - V2V3 - NE (TRANSFORMACIÓN DE COORDENADAS)
- estrellas brillantes (DETECCIÓN DE ESTRELLAS BRILLANTES)
- distancia del centroide a las estrellas más brillants (DISTANCIAS FÍSICAS A ESTRELLAS BRILLANTES)
- visualización con todo lo anterior, orientación, retícula, ejes... (VISUALIZACIÓN)


In [ ]:
# Librerías necesarias
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib.colors import LogNorm
from astropy.wcs import WCS
import math
from astropy import units as u
from scipy.ndimage import label, center_of_mass
from skimage.measure import regionprops
from skimage.feature import peak_local_max
from matplotlib.patches import Circle

# Cargar la fuente
font_path = "/usr/share/fonts/msttcore/times.ttf"
tnr_font = fm.FontProperties(fname=font_path)

# Cargar los datos de la imagen
F360M = fits.open('NGC4038/jw02581-o001_t001_nircam_clear-f360m_i2d.fits')  # FITS file
F360M_h = F360M[0].header
F360M_d = fits.getdata('NGC4038/jw02581-o001_t001_nircam_clear-f360m_i2d.fits')
F360M_d = np.nan_to_num(F360M_d)  # Reemplazar NaN por 0

# Obtener dimensiones
height, width = F360M_d.shape
print(f"Píxeles de la imagen: {F360M_d.shape}")

# Obtener WCS
wcs = WCS(F360M[1].header, naxis=2)

###################### PASO DE AR - h m s Y DEC º ' " ######################
# Obtener AR y DEC de la cabecera
AR = F360M_h['TARG_RA']
DEC = F360M_h['TARG_DEC']
# Función para convertir grados a horas, minutos, segundos (1h son 15º)
def grados_a_hms(ar_deg):
    horas = int(ar_deg / 15)  # Convertir grados a horas
    minutos = int((ar_deg / 15 - horas) * 60)  # Convertir la parte fraccionaria de horas a minutos
    segundos = ((ar_deg / 15 - horas) * 60 - minutos) * 60  # Convertir la parte fraccionaria de minutos a segundos
    return f"{horas}h {minutos}m {segundos:.2f}s"
# Convertir AR a formato de horas, minutos y segundos
AR_hms = grados_a_hms(AR)

# Función para convertir grados a grados, minutos y segundos
def grados_a_gms(deg):
    grados = int(deg)  # Parte entera de los grados
    minutos = int(abs(deg - grados) * 60)  # Convertir la parte decimal de grados a minutos
    segundos = (abs(deg - grados) * 60 - minutos) * 60  # Convertir la parte decimal de minutos a segundos
    return f"{grados}° {minutos}' {segundos:.2f}'' "

# Convertir DEC a formato de grados, minutos y segundos
DEC_gms = grados_a_gms(DEC)

##################################################################

#####################################   MÁSCARA PARA NÚCLEO   #####################################

#  Definir el radio de interés en píxeles para el núcleo
R_nucleo = 1450  # Reducimos el radio para centrarnos más en la zona brillante

#  Obtener el centro de la imagen
center_x, center_y = height // 2, width // 2

#  Crear una máscara circular para limitar la búsqueda del núcleo
Y, X = np.ogrid[:height, :width]
mask_circle_nucleo = (X - center_x) ** 2 + (Y - center_y) ** 2 <= R_nucleo ** 2  # Ecuación del círculo

#  Aplicar la máscara circular a la imagen original
masked_F360M = np.where(mask_circle_nucleo, F360M_d, 0)

#  Aplicar un umbral de brillo SOLO dentro del círculo del núcleo
umbral_nucleo = np.percentile(masked_F360M[mask_circle_nucleo], 95)  # Subimos el umbral para mejor precisión
mask_nucleo = masked_F360M > umbral_nucleo

#  Encontrar las regiones conectadas del núcleo
labeled, num_features = label(mask_nucleo)
region_sizes = np.bincount(labeled.ravel())[1:]  # Contar tamaños de regiones
largest_region_idx = np.argmax(region_sizes) + 1  # Índice de la región más grande
mask_nucleo = labeled == largest_region_idx  # Filtrar solo la región central

#  Obtener el centroide ponderado (núcleo de la nebulosa)
centroid = center_of_mass(masked_F360M * mask_nucleo)
centroid = (int(centroid[0]), int(centroid[1]))  # Convertir a enteros
print('---------------------------------------------------------------------------')
print(f"Centro refinado de NGC 4038: {centroid}")

# Convertir a coordenadas celestes
RA_centroid, DEC_centroid = wcs.all_pix2world([[centroid[1], centroid[0]]], 1)[0]
RA_centroid_hms = grados_a_hms(RA_centroid)
DEC_centroid_gms = grados_a_gms(DEC_centroid)
print(f"Centro refinado de NGC 4038 en coordendas (α,δ): ({RA_centroid_hms}, {DEC_centroid_gms}) ")
print('----------------------------------------------------------------------------')


#####################################  NÚCLEOS DEL SISTEMA    ########################################################
# --- Buscar los 2 puntos más brillantes ---
brightest_peaks = peak_local_max(F360M_d, min_distance=100, threshold_abs=np.percentile(F360M_d, 99.8), num_peaks=2)

# Obtener coordenadas
#centro1 = tuple(brightest_peaks[0])  # (y, x)
centro2 = tuple(brightest_peaks[1])  # (y, x)
centro2 = (int(brightest_peaks[1][0]), int(brightest_peaks[1][1]))

# Pasar núcleo 4039 a AR y DEC
RA_4039, DEC_4039 = wcs.all_pix2world([[centro2[1], centro2[0]]], 1)[0]
AR_4039_hms = grados_a_hms(RA_4039)
DEC_4039_gms = grados_a_gms(DEC_4039)

# Calcular distancia en píxeles
dist_pix = np.linalg.norm(np.array(centroid) - np.array(centro2))

# Escala de píxel (en arcsec/píxel)
pixel_scales = np.abs(wcs.pixel_scale_matrix.diagonal()) * 3600  # convertir a arcsec/pix
arcsec_per_pix = np.mean(pixel_scales)

# Distancia en arcsec
dist_arcsec = dist_pix * arcsec_per_pix

# Imprimir resultados
print("Coordenadas (y, x) núcleo 4038:", centroid)
print("Coordenadas (y, x) núcleo 4039:", centro2)
print(f"Coordenadas (α,δ) núcleo 4039: ({AR_4039_hms}, {DEC_4039_gms}) ")
print(f"Distancia entre núcleos: {dist_pix:.2f} px ≈ {dist_arcsec:.2f} arcsec")

#####################################   DISTANCIA ENTRE NÚCLEOS SABIENDO LA DISTANCIA AL SISTEMA GALÁCTICO   #####################################
# Calcular distancia real usando la aproximación: D = d * θ
# Convertir arcsec a radianes (1 arcsec = 1/206265 rad)

theta_rad = dist_arcsec / 206265  # sacamos el ángulo a partir de la distancia en arcsec de ambos núcleos

# Distancia al sistema (en años luz)
d_ly = 70000000  # 70 millones de años luz - información de internet

# Distancia real entre los núcleos en años luz
D_ly = d_ly * theta_rad

# Convertir a parsec 1ly = 0.3066pc
D_pc = D_ly * 0.3066

# Mostrar resultados
print(f"Distancia entre núcleos: {D_ly:.2f} ly ≈ {D_pc:.2f} pc")


#####################################   TRANSFORMACIÓN DE COORDENADAS   #####################################
# Datos de entrada
Xr = 5.77
Yr = 4.61
V2r = -89.40  # en arcosegundos
V3r = -491.36 # en arcosegundos

pixel_scales_deg = wcs.pixel_scale_matrix.diagonal()
pixel_scale = np.abs(pixel_scales_deg) * 3600
Sx, Sy = pixel_scale[0], pixel_scale[1]

beta_x = 0  # en grados
beta_y = 90  # en grados
X = 1020.0
Y = 1602.0
PA_V3 = 103.9690328145597  # Ángulo PA_V3 en grados

# Conversión de grados a radianes, para poder trabajar con ellos
beta_x_rad = math.radians(beta_x)
beta_y_rad = math.radians(beta_y)

# Convertir PA_V3 a radianes
PA_V3_rad = math.radians(PA_V3)

# Cálculo de V2 arcosegundos
V2 = V2r + Sx * math.sin(beta_x_rad) * (X - Xr) + Sy * math.sin(beta_y_rad) * (Y - Yr)

# Cálculo de V3 arcosegundos
V3 = V3r + Sx * math.cos(beta_x_rad) * (X - Xr) + Sy * math.cos(beta_y_rad) * (Y - Yr)

# Rotación de las coordenadas V2 y V3 al sistema Norte y Este
N = V3 * math.cos(PA_V3_rad) - V2 * math.sin(PA_V3_rad)  # Coordenada Norte relativa
E = V3 * math.sin(PA_V3_rad) + V2 * math.cos(PA_V3_rad)  # Coordenada Este relativa

print('----------------------------------------------------------------------------')
print('')

#####################################   DETECCIÓN DE ESTRELLAS BRILLANTES   #####################################
# Paso 1: Normalización (si no la tienes ya hecha)
img_norm = (F360M_d - np.min(F360M_d)) / (np.max(F360M_d) - np.min(F360M_d))

# Paso 2: Detección de picos locales (estrellas brillantes)
coords_estrellas = peak_local_max(img_norm, min_distance=5, threshold_abs=0.4)  # Puedes ajustar el umbral

# Mensaje informativo
print(f"Se han detectado y marcado las {len(coords_estrellas)} estrellas más brillantes.")

####################################   DISTANCIAS FÍSICAS A ESTRELLAS BRILLANTES  #####################################

# Distancia en años luz
d_ly = 70000000 # información sacada de internet
ly_to_pc = 0.3066
d_pc = d_ly * ly_to_pc  

# Escala de píxel (arcsec/pix)
pixel_scales = np.abs(wcs.pixel_scale_matrix.diagonal()) * 3600
arcsec_per_pix = np.mean(pixel_scales)

# Calcular distancias en píxeles desde el centroide
pix_distances = [np.linalg.norm(np.array([y, x]) - np.array(centroid)) for y, x in coords_estrellas]

# Pasar a arcsec
arcsec_distances = np.array(pix_distances) * arcsec_per_pix

# Pasar a radianes
theta_rad = (arcsec_distances / 3600) * (np.pi / 180)

# Distancias físicas en parsec
pc_distances = theta_rad * d_pc

# Mostrar 5 resultados más representativos (puedes ajustar el número)
print('\nDistancias desde el núcleo a las estrellas más brillantes:')
for i, (p, a, pc) in enumerate(zip(pix_distances[:5], arcsec_distances[:5], pc_distances[:5]), 1):
    print(f"Estrella {i}: {p:.1f} px ≈ {a:.2f}\" ≈ {pc:.2f} pc")
    
#####################################   VISUALIZACIÓN   #####################################
vmin = np.percentile(F360M_d[F360M_d > 0], 1)   # 5% más oscuro
vmax = np.percentile(F360M_d[F360M_d > 0], 99.9) # 99.5% más brillante
# Crear la figura sin distorsiones
fig = plt.figure(figsize=(9, 9))
ax = plt.subplot(projection=wcs)  ## Trabajar en 2D

#  Ajuste del colormap y el contraste
im = ax.imshow(F360M_d, cmap='inferno', norm=LogNorm(vmin=vmin, vmax=vmax))  # Ajustado para más contraste
ax.set_title('NGC 4038 - Sistema galáctico ''Las Antenas'' (F360M)', fontproperties=tnr_font, fontsize=16, pad=15)
ax.set_xlabel('δ', fontproperties=tnr_font, fontsize=12)
ax.set_ylabel('α', fontproperties=tnr_font, fontsize=12)
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax.coords[0].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # α (RA)
ax.coords[1].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Marcar el centro refinado en la imagen
ax.plot(centroid[1], centroid[0], '+', color='black', markersize=7, label="Centro NGC 4038")
ax.plot(centro2[1], centro2[0], '+', color='red', markersize=7, label="Centro NGC 4039")

# Dibujar círculos estrellas más brillantes y líneas desde el núcleo
for i, (y, x) in enumerate(coords_estrellas[:5]):  # solo las 5 más cercanas por claridad
    ax.add_patch(Circle((x, y), radius=10, edgecolor='lime', facecolor='none', linewidth=1.6))
    ax.plot([centroid[1], x], [centroid[0], y], color='orange', linestyle='--', linewidth=1.2)
    ax.text(x + 25, y, f'{i+1}', color='white', fontsize=13, fontproperties=tnr_font,
            ha='left', va='center', bbox=dict(facecolor='black', alpha=0.5, pad=1.5))

# Dibujar círculo alrededor de cada núcleo
circ1 = Circle((centroid[1], centroid[0]), radius=30, edgecolor='cyan', facecolor='none', linewidth=2, label='Núcleo 1')
circ2 = Circle((centro2[1], centro2[0]), radius=30, edgecolor='magenta', facecolor='none', linewidth=2, label='Núcleo 2')
ax.add_patch(circ1)
ax.add_patch(circ2)

# Distancia entre ambos núcleos del sistema
ax.plot([centroid[1], centro2[1]], [centroid[0], centro2[0]], color='white', linestyle='--', linewidth=1.5, label='Distancia entre núcleos')
ax.text((centroid[1] + centro2[1]) / 2, (centroid[0] + centro2[0]) / 2 - 120,
        'Distancia entre núcleos', color='white', fontsize=10, fontproperties=tnr_font,
        ha='center', va='bottom')



# Añadir anotación para AR y DEC
text_str = f"Coordenadas del núcleo\nα: {grados_a_hms(RA_centroid)}\nδ: {grados_a_gms(DEC_centroid)}"
ax.text(0.30, 0.95, text_str, transform=ax.transAxes, color='white', fontsize=12,
        fontweight='bold', fontproperties=tnr_font, ha='right', va='top', bbox=dict(facecolor='black', alpha=0.5))

# Añadir anotación para N y E
#text_str = f"Nc: {N:.6f} (arcsec)\nEc: {E:.6f} (arcsec)"
#ax.text(0.95, 0.85, text_str, transform=ax.transAxes, color='white', fontsize=10,
#        fontweight='bold', fontproperties=tnr_font, ha='right', va='top', bbox=dict(facecolor='black', alpha=0.5))


# --------------------------
# Añadir Norte y Este en la retícula correcta
# Punto en la esquina inferior derecha para la orientación
x, y = 1850, 215.5
ra_dec_corner = wcs.pixel_to_world(x, y)

# Desplazamientos para las flechas
delta_ra = (0.003 * u.deg)
delta_dec = (0.003 * u.deg)

# Calcular coordenadas de Norte y Este
ra_dec_norte = ra_dec_corner.spherical_offsets_by(0 * u.deg, delta_dec)
ra_dec_este = ra_dec_corner.spherical_offsets_by(delta_ra, 0 * u.deg)

# Convertir de vuelta a píxeles
norte_x, norte_y = wcs.world_to_pixel(ra_dec_norte)
este_x, este_y = wcs.world_to_pixel(ra_dec_este)

# Dibujar flechas en la dirección correcta
ax.arrow(x, y, norte_x - x, norte_y - y, 
         color='white', head_width=40, head_length=35, label="Norte")
ax.arrow(x, y, este_x - x, este_y - y, 
         color='white', head_width=40, head_length=35, label="Este")

# Etiquetas
ax.text(norte_x + 25, norte_y - 100, 'N', color='white', fontsize=13, fontproperties=tnr_font, ha='center', va='bottom')
ax.text(este_x - 45, este_y + 10, 'E', color='white', fontsize=13, fontproperties=tnr_font, ha='center', va='bottom')


# Añadir la barra de color
cbar = plt.colorbar(im, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)
cbar.set_label('Brillo', fontproperties=tnr_font, fontsize=12)
cbar.ax.tick_params(labelsize=10)
for label in cbar.ax.get_yticklabels():
    label.set_fontproperties(tnr_font)


# Configurar la retícula correctamente
ax.grid(color='white', linestyle='--', linewidth=0.5)
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)  # Ajuste del espacio

plt.show()
F360M.close()


### Base de código para realizar la composición RGB

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
import astropy.units as u
import matplotlib.font_manager as fm

# Fuente
font_path = "/usr/share/fonts/msttcore/times.ttf"
tnr_font = fm.FontProperties(fname=font_path)

# Cargar imágenes y WCS (asegurando abrir todos desde la misma carpeta)
F410M_hdu = fits.open('NGC4038/jw02581-o001_t001_nircam_clear-f410m_i2d.fits')  # Hα
F335M_hdu = fits.open('NGC4038/jw02581-o001_t001_nircam_clear-f335m_i2d.fits')  # [O III]
F150W_hdu = fits.open('NGC4038/jw02581-o001_t001_nircam_clear-f150w_i2d.fits')  # He II

F410M = np.nan_to_num(F410M_hdu[1].data)
F335M = np.nan_to_num(F335M_hdu[1].data)
F150W = np.nan_to_num(F150W_hdu[1].data)
wcs = WCS(F410M_hdu[1].header)  # Usamos WCS de la primera imagen, por ejemplo

# Igualar tamaños
min_shape = (min(F410M.shape[0], F335M.shape[0], F150W.shape[0]), 
             min(F410M.shape[1], F335M.shape[1], F150W.shape[1]))

F410M_crop = F410M[:min_shape[0], :min_shape[1]]
F335M_crop = F335M[:min_shape[0], :min_shape[1]]
F150W_crop = F150W[:min_shape[0], :min_shape[1]]

# Normalización individual
def normalize_img(img):
    img_clip = np.clip(img, np.percentile(img[img > 0], 5), np.percentile(img[img > 0], 99))
    norm = (img_clip - img_clip.min()) / (img_clip.max() - img_clip.min())
    return norm

R = normalize_img(F410M_crop)
G = normalize_img(F335M_crop)
B = normalize_img(F150W_crop)

# Crear imagen RGB
rgb_image = np.zeros((R.shape[0], R.shape[1], 3))
rgb_image[:, :, 0] = R
rgb_image[:, :, 1] = G
rgb_image[:, :, 2] = B

# Función para flechas N y E
def N_E_arrows(ax, wcs):
    x, y = 1850, 170
    ra_dec_corner = wcs.pixel_to_world(x, y)
    delta_ra = 0.002 * u.deg
    delta_dec = 0.002 * u.deg
    ra_dec_norte = ra_dec_corner.spherical_offsets_by(0 * u.deg, delta_dec)
    ra_dec_este = ra_dec_corner.spherical_offsets_by(delta_ra, 0 * u.deg)
    norte_x, norte_y = wcs.world_to_pixel(ra_dec_norte)
    este_x, este_y = wcs.world_to_pixel(ra_dec_este)

    ax.arrow(x, y, norte_x - x, norte_y - y, color='red', head_width=20, head_length=25)
    ax.arrow(x, y, este_x - x, este_y - y, color='red', head_width=20, head_length=25)
    ax.text(norte_x + 10, norte_y - 80, 'N', color='red', fontsize=15, fontproperties=tnr_font, ha='center')
    ax.text(este_x - 40, este_y, 'E', color='red', fontsize=15, fontproperties=tnr_font, ha='center')

print(f"Píxeles de la imagen: {rgb_image.shape}")

# Mostrar subplots con cada filtro y la combinación
fig = plt.figure(figsize=(20, 15))
plt.subplots_adjust(wspace=0.4)  # Aumenta la separación horizontal

# Hα
ax1 = fig.add_subplot(2, 3, 1, projection=wcs)
ax1.imshow(R, cmap='Reds')
ax1.set_title("F410M (polvo más frío)", fontproperties=tnr_font, fontsize=24)
ax1.grid(color='gray', linestyle='--', linewidth=0.5)
N_E_arrows(ax1, wcs)
ax1.set_xlabel('δ', fontproperties=tnr_font, fontsize=18)
ax1.set_ylabel('α', fontproperties=tnr_font, fontsize=18)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax1.coords[0].set_ticklabel(size=26, fontproperties=tnr_font, simplify=False)  # α (RA)
ax1.coords[1].set_ticklabel(size=26, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon1 = ax1.coords[0]  # Eje de Ascensión Recta (α)
lat1 = ax1.coords[1]  # Eje de Declinación (δ)

# Forzar la visualización completa de horas, minutos y segundos en α
lon1.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat1.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon1.display_minor_ticks(True)
lat1.display_minor_ticks(True)
#############################################################################



# [O III]
ax2 = fig.add_subplot(2, 3, 2, projection=wcs)
ax2.imshow(G, cmap='Greens')
ax2.set_title("F335M (polvo caliente)", fontproperties=tnr_font, fontsize=24)
ax2.grid(color='gray', linestyle='--', linewidth=0.5)
N_E_arrows(ax2, wcs)
ax2.set_xlabel('δ', fontproperties=tnr_font, fontsize=18)
ax2.set_ylabel('α', fontproperties=tnr_font, fontsize=18)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax2.coords[0].set_ticklabel(size=26, fontproperties=tnr_font, simplify=False)  # α (RA)
ax2.coords[1].set_ticklabel(size=26, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon2 = ax2.coords[0]  # Eje de Ascensión Recta (α)
lat2 = ax2.coords[1]  # Eje de Declinación (δ)

# Forzar la visualización completa de horas, minutos y segundos en α
lon2.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat2.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon2.display_minor_ticks(True)
lat2.display_minor_ticks(True)
#############################################################################


# He II
ax3 = fig.add_subplot(2, 3, 3, projection=wcs)
ax3.imshow(B, cmap='Blues')
ax3.set_title("F150W (estrellas jóvenes)", fontproperties=tnr_font, fontsize=24)
ax3.grid(color='gray', linestyle='--', linewidth=0.5)
N_E_arrows(ax3, wcs)
ax3.set_xlabel('δ', fontproperties=tnr_font, fontsize=18)
ax3.set_ylabel('α', fontproperties=tnr_font, fontsize=18)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax3.coords[0].set_ticklabel(size=26, fontproperties=tnr_font, simplify=False)  # α (RA)
ax3.coords[1].set_ticklabel(size=26, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon3 = ax3.coords[0]  # Eje de Ascensión Recta (α)
lat3 = ax3.coords[1]  # Eje de Declinación (δ)


# Forzar la visualización completa de horas, minutos y segundos en α
lon3.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat3.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon3.display_minor_ticks(True)
lat3.display_minor_ticks(True)
#############################################################################


# RGB combinado
ax4 = fig.add_subplot(2, 3, 5, projection=wcs)
ax4.imshow(rgb_image)
ax4.set_title("NGC 4038 - Composición RGB (A)", fontproperties=tnr_font, fontsize=24)
ax4.grid(color='white', linestyle='--', linewidth=0.5)
N_E_arrows(ax4, wcs)
ax4.set_xlabel('δ', fontproperties=tnr_font, fontsize=18)
ax4.set_ylabel('α', fontproperties=tnr_font, fontsize=18)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax4.coords[0].set_ticklabel(size=26, fontproperties=tnr_font, simplify=False)  # α (RA)
ax4.coords[1].set_ticklabel(size=26, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon4 = ax4.coords[0]  # Eje de Ascensión Recta (α)
lat4 = ax4.coords[1]  # Eje de Declinación (δ)

# Forzar la visualización completa de horas, minutos y segundos en α
lon4.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat4.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon4.display_minor_ticks(True)
lat4.display_minor_ticks(True)
#############################################################################


plt.tight_layout()
plt.show()


### Base de código para realizar los perfiles de brillo

In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

# Cargar los filtros RGB de la composición A
F410M = np.nan_to_num(fits.getdata('NGC4038/jw02581-o001_t001_nircam_clear-f410m_i2d.fits'))  # R
F335M = np.nan_to_num(fits.getdata('NGC4038/jw02581-o001_t001_nircam_clear-f335m_i2d.fits'))  # G
F150W = np.nan_to_num(fits.getdata('NGC4038/jw02581-o001_t001_nircam_clear-f150w_i2d.fits'))  # B

# Igualar tamaño
min_shape = (min(F410M.shape[0], F335M.shape[0], F150W.shape[0]),
             min(F410M.shape[1], F335M.shape[1], F150W.shape[1]))
F410M = F410M[:min_shape[0], :min_shape[1]]
F335M = F335M[:min_shape[0], :min_shape[1]]
F150W = F150W[:min_shape[0], :min_shape[1]]

# Centro definido manualmente para NGC 4038
center_x, center_y = 1020, 1603

# Suavizado opcional (mejor para perfiles suaves)
F410M_smooth = gaussian_filter(F410M, sigma=1.5)
F335M_smooth = gaussian_filter(F335M, sigma=1.5)
F150W_smooth = gaussian_filter(F150W, sigma=1.5)

# Calcular perfil radial
def perfil_radial(imagen, centro, r_max):
    y, x = np.indices(imagen.shape)
    r = np.sqrt((x - centro[1])**2 + (y - centro[0])**2).astype(int)
    perfil = np.zeros(r_max)
    for i in range(r_max):
        mask = (r == i)
        if np.sum(mask) > 0:
            perfil[i] = np.mean(imagen[mask])
    return perfil

# Radios hasta los que analizar (ajusta si quieres más lejos)
r_max = 100

# Calcular perfiles para R, G, B
perfil_R = perfil_radial(F410M_smooth, (center_x, center_y), r_max)
perfil_G = perfil_radial(F335M_smooth, (center_x, center_y), r_max)
perfil_B = perfil_radial(F150W_smooth, (center_x, center_y), r_max)

# --- PLOT ---
plt.figure(figsize=(8,6))
plt.plot(perfil_R, color='red', label='(emisión térmica polvo caliente - más difuso) - F410M (R)')
plt.plot(perfil_G, color='green', label='(emisión térmica polvo caliente) - F335M (G)')
plt.plot(perfil_B, color='blue', label='(emisiones centradas en NIR) - F150W (B)')
plt.title("Perfil Radial - Composición RGB A (NGC 4038)")
plt.xlabel("Distancia radial (pixeles)")
plt.ylabel("Brillo promedio")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
